# Spacial boxcount algorithm and translation into convolutional neural network

Ole Peters
    
## 1 Abstract

This paper contains the postulation of a spacial boxcount algorithm, which characterizes any incoming 2D array spacially by indicators for topological complexity and spacial heterogeneity at different scales. This characterization allows spacial similarity search or sorting capability, edge detection with userspecified scaling and statistical analysis of input datasets. The algorithm can handle any kind of data able to be represented in a 2 dimensional array. 
By training a convolutional neural network to mimic the cpu driven function, the process could speedup by a huge factor utilizing the parallel capabilities of a graphics card.

## 2 Introduction

The daily advances in machine learning establish technological advances in almost any area of research. 
Convolutional neural networks gained a lot of traction, outperforming linear models and even recurrent models in timeseries forecasting. They often are used in object detection and are computationally intensive tasks.

Box counting like [1] is a method to analyze data by breaking the input array into boxes at different scales and counting every filled box indicating the spacial complexity for each scaling by the chosen the boxsize. An animation of this process is shown in figure 1.

![BoxcountingURL](https://upload.wikimedia.org/wikipedia/commons/5/53/Fixedstack.gif "Fixed grid scans")
Figure 1: Fixed grid gliding box scanning animation
Source: <https://commons.wikimedia.org/wiki/File:Fixedstack.gif>

Lacunarity is a mathematical discription for spacial heterogeneity at a chosen scale [2].
With the boxcount and lacunarity, which depends on the boxsize, comparisons between different samples of the same data type can be made. By comparing these samples, similar structures can be found and sorted by spacial complexity and heterogeneity. This enables characterization of continuously distributed data.

![LacunarityUrl](https://upload.wikimedia.org/wikipedia/commons/3/31/Rotational_Invariance_Example.gif "lacunarity")
Figure 2 : Lacunaritys or spacial heterogeneity of three different patterns
Source: https://commons.wikimedia.org/wiki/File:Rotational_Invariance_Example.gif


The spacial resolution of the boxcount algorithm can be achieved by chunking the picture into boxsize specified large areas, where within the boxcounting is executed resulting in a 2D array scaled inverse by the boxsize. These arrays represent the spacially distributed topological complexity and heterogeneity at a chosen scale. The counted boxes reveal different topological features and the lacunarity indicates defects in pattern respectively to their boxsizes. The boxsizes 2, 4, 8 and 16 were used, resulting in arrays with corresponding areas of 1/2, 1/4, 1/8, 1/16 of the original image size.

The example dataset consists of electron microscope images of femtosecond laser machined metal surfaces like the example shown in figure 3. These micro- and nanostructures are laser induced periodic surface structures, which are also known as LIPSS and can achieve drastic alterations on surface wettability.
Also a few images of hydrophobic leaf surfaces are contained in the dataset.

The dataset can be downloaded at: <https://drive.google.com/file/d/1sLc8Uk61W_4TTtmRk4x6_5cOiWq7IeOT/view?usp=sharing>

![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/MISC/17_3_Rand.bmp)

Figure 3: Sample photo of a femtosecond laser machined metal surface with various surface features

In Figure 4 an output of the program has been generated, which results in pictures composed of boxcount ratios and lacunaritys at the chosen boxsize.
Scale dependent features emerge at different boxsizes. The boxcount array indicates topological complexity, while contrast in the lacunarity array can distinguish spacial heterogeneous from homogenous surface featuers. 

![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/generated_imgs/17_3_Rand.png)

Figure 4: Generated picture of resulting boxcount and lacunarity arrays with the boxsizes 2,4,8,16  reveal different subpatterns derived from the surface structure.

The concept of scanning arrays to count boxes or to perform a computation, like calculating the lacunarity, compared to the concept of convolutional neural networks with their kernel sized, stride moving convolution layers seem very similar. So a convolutional neural network could learn this task utilizing parallel processing power.

![Convolution_Animation](https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif "Convolution Animation")

Figure 5: Animation of a convolution operation in a convolutional neural network.   
Source: https://commons.wikimedia.org/wiki/File:2D_Convolution_Animation.gif


## 3 Experimental Setup and Methods

In this paper a boxcount algorithm is proposed which can characterize data spacially in topological complexity and heterogeneity at a chosen scale.

This algorithm enables characterizing, sorting and searching for spacial similarity for any kind of input data. Here pictures of different topologies formed in a process of laser induced periodic surface structures(LIPSS) were used to demonstrate the algorithms capability.


### 3.1 Prerequisites
Python3 (preferred version <=3.8) is used to execute this code. By using version 3.8, the essential package named numba is not available at date of publication.
The following external packages can be installed with the module manager pip, which is already included in python.
To handle n-dimensional array data efficiently the numpy module is necessary and many other imports depend on numpy to function correctly.
The module pillow allows to load pictures in many different formats and handles image manipulation, like cropping and accessing individual channels, like RGB, RYB.
To visualize the progress during tasks, the module tqdm displays a progressbar with other statistics by a modified for loop. Matplotlib is used to make graphs and plots from arrays, data or pictures.
In case of high level path accessing capabilities pathlib is needed and for storing and loading arbitrary data pickel is used.
While developing an algorithm with python due its simple code structure is rather fast, executing performance compared to other languages like c,c++,c# or assembler is terrible. In this case the numba jit compiler compiles decorated python functions in c code which speeds up process intensive tasks by a huge margin.

For the machine learning part pytorch in combination of a graphics card and the necessary cuda package is used to speed up training time. For testing and executing a model just the cpu and ram is needed, but the gpu can also be utilized to speed up computation and is neccesary due to long training times. To preprocess incoming data the scykit-learn package is utilized. For hyperparameter optimization the package hyperopt was chosen to suggest values for various variables/hyperparameters to achieve a minimized loss during training.

In [6]:
#ATTENTION: This cell has only to be executed, if you want to install the necessary packages.
#           So just execute, if you want to install the package.

#Update Python Package Manager pip
!python3 -m  pip install --upgrade pip --user
#Prequesits linux via Python Package installer pip:
#Python version 3 has to be installedon the system to be able to execute the code 
!pip3 install numpy pillow tqdm matplotlib pathlib


#ATTENTION if you use python3.9 numba will not work currently, so comment every line with 'numba' and 'jit' out
!pip3 install numba

# install pytorch with gpu support for local cuda version 11.0... modify as needed 
#  [15] https://varhowto.com/install-pytorch-1-6-0/
'''
ATTENTION: FOR CUDA-GPU VERSION USE THE NEXT LINE, IF CPU IS WANTED, 
JUST COMMENT THE NEXT OUT AND  UNCOMMENT THE CPU INSTALL COMMAND in 3 LINES
'''
#!pip3 install torch===1.7.0+cu110 torchvision===0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
#  if there is no CUDA capable GPU just install cpu version

#!pip3 install torch==1.7.1+cpu torchvision==0.8.2+cpu -f https://download.pytorch.org/whl/torch_stable.html
    
# when this versions doesn't work, try some other versions from the previous link

!pip3 install sklearn hyperopt # install sci-kit learn and hyperoptimization libary

'''
Code package documentation:
[3] https://numpy.org/doc/1.20/
[4] https://tqdm.github.io/docs/tqdm/
[5] https://matplotlib.org/3.3.3/contents.html
[6] https://docs.python.org/3/library/pathlib.html
[7] https://numba.readthedocs.io/en/stable/user/jit.html
[8] https://pytorch.org/docs/stable/nn.init.html
[9] https://sklearn.org/documentation.html
[10] http://hyperopt.github.io/hyperopt/
'''

ERROR: Invalid requirement: '#'


'\nCode package documentation:\n[3] https://numpy.org/doc/1.20/\n[4] https://tqdm.github.io/docs/tqdm/\n[5] https://matplotlib.org/3.3.3/contents.html\n[6] https://docs.python.org/3/library/pathlib.html\n[7] https://numba.readthedocs.io/en/stable/user/jit.html\n[8] https://pytorch.org/docs/stable/nn.init.html\n[9] https://sklearn.org/documentation.html\n[10] http://hyperopt.github.io/hyperopt/\n'

## 3.2 Imports

To execute the code a few necessary python modules and libraries need to be loaded. The imported modules range from external libraries, which were discussed in 3.1 to internal modules like os and time. Also linecache and sys are used to generate a helper function to print information from thrown exceptions, executed in a "try and except" syntax. 
Matplotlib is used to display arrays as pictures with the specified helper function executed with the corresponding array, title and colormap. Documentations of all used python libraries are found in chapter 6 at [3-14].

In [2]:
#Imports------------------------------------------------------------------------------------------
import os              #Operating Sytem Module for getting paths, commands, etc...
from os import listdir #class for listing files in a directory
import time            #for time measurement
import pickle          #for loading/saving Data
from PIL import Image  #Imagemanipulation via pillow image module

import pathlib              #Import pathlib to create a link to the directory where the file is at.
#just has to be specified in jupyter, if executed via the terminal __file__ is been found
__file__ = 'Spacial boxcount algorithm CPU and GPU.ipynb'    # Just use this when  __file__ is not been found
FileParentPath = str(pathlib.Path(__file__).parent.absolute()) # Variable for path, where this file is in!
'''
Code package documentation:
[11] https://docs.python.org/3/library/os.html
[12] https://docs.python.org/3/library/pickle.html
[13] https://docs.python.org/3/library/time.html
[14] https://pillow.readthedocs.io/en/stable/
'''
#Math module imports----------------------------
import numpy as np     # Python package for numeric computation

#Visualizing
import matplotlib.pyplot as plt # Python plotting libary for visualizing graphs, data and images
import matplotlib.image as img 
#%matplotlib inline  #just used in jupyter, if the plots are not shown
from tqdm import tqdm  # Progressbar

#Importing nessecary modules for creating machine learning networks, such as 
import torch                                    # Pytorch machine learning framework.
import torch.nn as nn                           # neural network components
import torch.nn.functional as F                 # activation functions
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.optim as optim                      #optimizer here adam will be used

#Preprocessing libary for scaling and normalzing nD-arrays
import sklearn.preprocessing as preprocessing

#Hyperparameteroptimization for training network with coustom parameters
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials  #hyperoptimization libary


#If something goes wrong just use the PrintExeption() function to get all neccesary data
#Sourced from: [16] https://stackoverflow.com/questions/14519177/python-exception-handling-line-number#20264059
import linecache
import sys

def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))
    
    
#Helper-function to show any np.array as a picture with a chosen title and a colormapping 
def showNPArrayAsImage(np2ddArray, title, colormap):
    plt.figure()                    #Init figure
    plt.imshow(np2ddArray,          #Gererate a picture from np.array and add to figure
            interpolation='none',
            cmap = colormap)
    plt.title(title)                #Add title to figure
    plt.show(block=False)           #Show array as picture on screen, but dont block the programm to continue.


verbosity = False   # False: no text displayed ;  True: potentially useful printouts are shown for info/bugfixing.


## 3.3 Spacial Boxcount CPU algorithm


### 3.3.1 Function: Z-boxcounting - Intrinsic value boxcounting

The z-boxcounting is derived from the original boxcount concept [1]. The function is executed with a 2D, picture like array with intrinsic values ranging depending on the given data format. The maximum value of the intrinsic dimensionality depends on the numeric encoding of the data. The maximum intrinsic value for hexcode is 15, for a normal 8-bit grayscale image it is 255, while a high dynamic range picture with 12 bit has a value of 4095. 
The z-boxcounting gets a chunk of the original picture or array so x and y borders are defined externally.
The input array is now seen as a volume with lateral borders (x, y) and the height defined by the maximum intrinsic value into the z direction, hence the name z-boxcounting. The algorithm counts the sum of all, at least partially, filled boxes in a fixed grid scan scaled by the specified boxsize. The actual count ranges inversely proportional to its chosen boxsize and can get really high with a 2³ box. To get the same value range for each boxsize, the counted boxes are divided by the total number of boxes resulting in the boxcount ratio. This ratio ranges from zero, with no counted boxes, to one, describing that all possible boxes were counted.

![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/Document_images/lacunarity_formula.png)
Figure 6: Formula to calculate the lacunarity, given the standard deviation sigma and mean µ.  

At the same time a list of counted values is recorded per box, so the lacunarity, or spacial heterogeneity can be calculated. The lacunarity is calculated by taking the standard deviation σ of the list of counted values for each box and is devided by the mean µ of it [2]. So that all output lacunaritys are positive the former term is taken to the power of 2. This results in a property where lacunaritys against zero describing a homogeneous structures and by increasing lacunarity spacial heterogeneity emerges.

The input spacial data chunk ranging in x,y and intrinsic value will be transformed into statistical data. This statistical data can be compared to other boxcounted structures.



![BoxcountingURL](https://upload.wikimedia.org/wikipedia/commons/5/53/Fixedstack.gif "Fixed grid scans")
Figure 7: Fixed grid scanning animation
Source: https://commons.wikimedia.org/wiki/File:Fixedstack.gif


### 3.3.2 Function: Spacial boxcounting

The spacial boxcounting is cropping the input picture, or 2D numpy array to the specified box boundries in x and y respectively. This cropped box represents the gliding box, which is scanning in a fixed grid manner iterativly over the picture or array and determines the counted boxes and the lacunarity at a given coordinate in the picture by accessing the z-boxcounting function. The boxsize of is equal to the stride resulting in a fixed grid with no overlap slicing the input array into chunks. The z-boxcounting is applied to each chunk and returns the boxcount ratio and lacunarity, which forms a boxcount and lacunarity array individually according to the spacial location in the original image and is scaled down by the inverse of the boxsize. 


### 3.3.3 C-compiler and multithreading for optimizing computation time

#### 3.3.3.1 C-compiler jit from numba
To gain huge speedup compared to the standard python compiler a different compiler can be used just by decorating the specified function with a "@jit(nopython=True)" decorator. The compiling of the c-code results in an increased process overhead, but a significant speedup in computation. For time comparison, go to section 4.2 in results and discussion. The numba compiler can not translate every arbitrary python function from every module, so the code has to be syntax conform by using simple python or numpy functions to achieve proper results.


#### 3.3.3.2. Multithreading
The boxcounting is a sequential task, which has to be rerun for each chosen boxsize and it results in four parallelizable tasks. This parallelization can be utilized by the threading module and by defining a thread-class, which is able to return a value, however this is not achievable with the threading base-class.
Each thread with it's chosen boxsize is started, executed and after the completion it's rejoined with the main thread to create a dictionary with all spacial boxcount- and lacunarity-arrays.

In [3]:
verbosity = False   # False: no text displayed ;  True: potentially useful printouts are shown for info/bugfixing.

#Numba translates Python functions to optemized machine code at runtime and results in significant speedups
from numba import jit

#If you want to compare the jit-compiler to the standard python interpreter just un/comment all @jit(... lines.
@jit(nopython= True,nogil=True) # Set "nopython" mode for best performance, equivalent to @njit
def Z_boxcount(GlidingBox, boxsize,MaxValue):
    continualIndexes = GlidingBox/boxsize    # tells in which boxindex the given value is in a continual way
    Boxindexes = np.floor(continualIndexes)  # Round all boxindexes down to ints to get the boxindex of each value in the gliding box 
    
    # If a value is in a given box, the boxcount increases by 1, but no further, when another  value is in the same box
    unique_Boxes = np.unique(Boxindexes)    # numpy helper function to create a list of the unique values by discarding doubles 
    counted_Boxes = len(unique_Boxes)       # the lenght of the list of all unique indexes are all unique counted boxes within the gliding box and the z-range

    if verbosity == True:
        print(continualIndexes)
        print("Boxindex",Boxindexes)
        print("counted_Boxes",counted_Boxes)
    
    #CREATE  List of SumPixInBox for all boxes to calc lacunarity
    InitalEntry = [0.0]
    SumPixInBox = np.array(InitalEntry)

    #For tiny boxes it can be process consuming
    #for every unique counted boxindex in the list of all unique boxindexes
    for unique_BoxIndex in unique_Boxes:
        #set element-wise to True, when a boxindex is equal to the chosen unique boxindex
        ElementsCountedTRUTHTABLE = Boxindexes == unique_BoxIndex
        #the sum of the True elements represent the count of datapoints/pixel/voxel within the chosen box
        ElementsCounted = np.sum(ElementsCountedTRUTHTABLE)
        #Append the list of ElementsCounted to the list of all Elementcounted-lists to calc lacunarity later
        SumPixInBox = np.append(SumPixInBox, ElementsCounted)
        
        if verbosity == True:
            print("unique_BoxIndex",unique_BoxIndex)
            print("ElementsCountedTRUTHTABLE",ElementsCountedTRUTHTABLE)
            print("ElementsCounted",ElementsCounted)

            
    # Because the lacunarity is calculated with the standard deviation of all elementscounted
    # and the not counted boxes have a value of 0 boxes counted in this box
    # we have to calc the number of empty boxes by subtracting all counted boxes from the total amount of possible boxes
    Max_Num_Boxes = int(MaxValue/ boxsize)
    Num_empty_Boxes = Max_Num_Boxes- counted_Boxes
    
    if Num_empty_Boxes <1:      
        pass
        # if there is are no empty boxes, just pass
    else:
        EmptyBoxes = np.zeros(Num_empty_Boxes)
        SumPixInBox = np.append(SumPixInBox, EmptyBoxes)
    # calcs the mean of the list of all counted datapoints/pixel/voxel within chosen boxes and then...
    mean = np.mean(SumPixInBox)	 # ...calcs the standard deviation of the same
    standardDeviation = np.std(SumPixInBox)
    #The lacunarity or spacial heterogeneity  = (standard deviation/mean)^2 
    Lacunarity=np.power(standardDeviation/mean,2)

    if verbosity == True:
        print("Max_Num_Boxes",Max_Num_Boxes)
        print("Num_empty_Boxes",Num_empty_Boxes)
        print("mean",mean)
        print("standardDeviation",standardDeviation)
        print("die Lacunarity ist", Lacunarity)

    return counted_Boxes, Lacunarity



@jit(nopython= True, nogil=False) #False,forceobj=True) # Set "nopython" mode for best performance, equivalent to @njit
def spacialBoxcount(npOutputFile, iteration,MaxValue):
    '''
    This function takes in a 2D np.array the iteration which determins the boxsize
    and the maximum possible value to set up the value range. 8-Bit -> 256, hexadez ->16
    
    The function returns a 2 channel-2d array containing the spacial boxcount ratio and the
    spacial lacunarity scaled down in size by 1/Boxsize[iteration]
    '''
    Boxsize=[2,4,8,16,32,64,128,256,512,1024]  #All boxsizes
    boxsize = Boxsize[iteration]               #specified boxsize

    #Init counting box at x=0,y=0 und z=0
    BoxBoundriesX = np.array([0,Boxsize[iteration]])
    BoxBoundriesY = np.array([0,Boxsize[iteration]])

    Boxcount = 0
    YRange, XRange  = npOutputFile.shape
    
    #The maximum index of box with given boxsize in x and y direction
    maxIndexY = YRange / boxsize 
    maxIndexY = int(maxIndexY)+1

    maxIndexX = XRange / boxsize 
    maxIndexX = int(maxIndexX)+1

    
    
    if verbosity == True:
        print( "XRange, YRange", XRange, YRange)
        print("maxIndexX: ",maxIndexX,"maxIndexY: ",maxIndexY)

    #Initialize the BoxcountRatio_map and spacial_lacunarity_map with zeros in correct shape
    BoxCountR_map = np.zeros((maxIndexY,maxIndexX))
    spa_Lac_map = np.zeros((maxIndexY,maxIndexX))


    while BoxBoundriesY[1]<=YRange:

        while BoxBoundriesX[1]<=XRange:
            #Set up Boxindex with boxsize
            indexY = int(BoxBoundriesY[0]/boxsize)
            indexX = int(BoxBoundriesX[0]/boxsize)
            
            #Define Gliding box with Boundries ... for ex.  Boxsize 4  -> Boxboundries [0,4],[4,8] -> geht auf für n² wie in bildern etc und stitching is möglich
            GlidingBox = npOutputFile[BoxBoundriesY[0]:BoxBoundriesY[1],BoxBoundriesX[0]:BoxBoundriesX[1]] 

            counted_Boxes, Lacunarity = Z_boxcount(GlidingBox, boxsize, MaxValue)
            
            #Despite counting the Boxes like in the original algorithm, the counts are normalized from 0...1
            #0 means there was nothing counted inside  and 1 means every possible box is filled in
            Max_Num_Boxes = int(MaxValue/ boxsize)
            counted_Box_Ratio = counted_Boxes / Max_Num_Boxes 
            
            BoxCountR_map[indexY,indexX] = counted_Box_Ratio
            spa_Lac_map[indexY,indexX] = Lacunarity

            #move box into x direction, while boxboundriesx are <= xrange
            BoxBoundriesX[0]+=Boxsize[iteration]
            BoxBoundriesX[1]+=Boxsize[iteration]
            
            if verbosity == True:
                print("indexX: ",indexX)
                print("indexY: ",indexY)
                print("BoxBoundriesX: ",BoxBoundriesX)
                print("BoxBoundriesY: ",BoxBoundriesY)
                print("GlidingBox: ", GlidingBox)
                print("counted_Boxes, Lacunarity.: ",counted_Boxes, Lacunarity)

                
        #By exit inner while loop, box has reached end of x axis in array, so reset boxboundriesX to start
        BoxBoundriesX[0]=0
        BoxBoundriesX[1]=Boxsize[iteration]
        #and increase the counting box in y direction by a boxsize to scan the next line
        BoxBoundriesY[0]+=Boxsize[iteration]
        BoxBoundriesY[1]+=Boxsize[iteration]

    BoxCountR_SpacialLac_map = [BoxCountR_map, spa_Lac_map]
    
    if verbosity == True:
        print(BoxCountR_map)
        print(spa_Lac_map)
        print("Iteration ", iteration, "calculation done")

    return BoxCountR_SpacialLac_map





def MultithreadBoxcount(npOutputFile):
    
    '''
    To gain another speedup in the sequential generated output, multi threading is used
    to calculate the spacial Boxcountratios/lacunaritys for each boxsize in a own thread.
    
    '''
    #MULTICORE APROACH
    #print("Beginn Multithread Boxcount Lacunarity feature extraction")
    BoxsizeDict={"2":0 ,"4":1,"8":2,"16":3,"32":4,"64":5,"128":6,"256":7,"512":8,"1024":9}

    #Cut to lenght
    Height , width = npOutputFile.shape
    Height , width = int(Height) , int(width) 
    BaseITERMinVal = min(16,Height , width  )
    BaseIteration = BoxsizeDict[str(int(BaseITERMinVal))] #without 0 there are 1 more processes 
    maxiteration =  BaseIteration +1    # to calc Lacunarity there have to be more than just one box into the z direction
    
    #source: [17]  https://stackoverflow.com/questions/6893968/how-to-get-the-return-value-from-a-thread-in-python

    def BoxcountBoxsizeWorker(npOutputFile, iteration):
        maxvalue = 256  # cause zheight is 0...255: 8-bit grayscale picture
        #adjust for every specific input
        BoxCountR_SpacialLac_map = spacialBoxcount(npOutputFile, iteration,maxvalue )     
        
        return BoxCountR_SpacialLac_map

    from threading import Thread
    
    #Create thread-class with ability to return a value, which is not possible in threading
    class ThreadWithReturnValue(Thread):
        def __init__(self, group=None, target=None, name=None,
                    args=(), kwargs={}, Verbose=None):
            Thread.__init__(self, group, target, name, args, kwargs)
            self._return = None
        def run(self):
            #print(type(self._target))
            if self._target is not None:
                self._return = self._target(*self._args,
                                                    **self._kwargs)
        def join(self, *args):
            Thread.join(self, *args)
            return self._return


    #Init number of needed threads
    threads = [None] * maxiteration      
    if verbosity: print("Generate ",maxiteration,"threads")

    start = time.time()

    for i in range(len(threads)):
        threads[i] = ThreadWithReturnValue(target=BoxcountBoxsizeWorker, args=(npOutputFile, i))
        threads[i].start()
        if verbosity == True:
            print("thread ",i+1," has started")

    BoxCountR_SpacialLac_map_Dict = {"iteration": np.array(["BoxcountRatio","spacialLacunarity"]) }
    for i in range(len(threads)):
        BoxCountR_SpacialLac_map = np.array(threads[i].join())
        BoxCountR_SpacialLac_map_Dict[i]= BoxCountR_SpacialLac_map
        if verbosity == True :
            #print(BoxCountR_SpacialLac_map)
            print(BoxCountR_SpacialLac_map.shape)
            print(type(BoxCountR_SpacialLac_map))
            print("Thread ",i," JOINED")

    end = time.time()

    print(round(end - start,3),"seconds for spacial boxcounting with ",i+1, "iterations/scalings")
    
    if verbosity:
        input("Press any key to continue with next file. \n Attention: verbosity adds much size to the output of jupyter notebook. If the file > 120'ish MB, jupyter notebook crashes. So use just for debugging for beware. ")
    
    return BoxCountR_SpacialLac_map_Dict


### 3.3.4 Validation of performance - visualizing and saving processed images with spacial boxcounting


By iterating over a picture containing folder and measuring the time difference of the multi threaded boxcounting function, the standard python compiler can be compared to the numba-jit compiler in terms of computational efficiency. 

If the variable verbosity is set to true, additional information and the generated arrays are displayed, but can eventually buffer overflow the output of the jupyter notebook file, resulting in a unaccessible file.
The following function computes the first 100 pictures specified by a foldername, tracks the time for executing the multithreaded boxcounting and displays corresponding outputs to visualize the spacial boxcounting.
If the folder name "MISC" is chosen, the images from the chosen folder is processed and by user input can be specified, if the calculated arrays are to be saved in the folder generated images like shown in figure 8.

![BoxcountingURL](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/generated_imgs/17_3_900x.png)
Figure 8: Example generated image with the boxcounting ratios (BCR) and lacunaritys (LAC) to their boxsize (2,4,8,16)

In [4]:
verbosity = False

foldername = "Images" # The foldername  where the machine learning dataset is based on
#foldername = "MISC"   # for saving special fotos iterate over MISC
whereTObreakIteration = 100 #abort after 100 pictures for time testing

 

def Show_Pictures(foldername):
    savepicture = None
    display_images = input("(Y/n) Do you want to display the processed images? Attention: Output gets big quickly and can resolve in a soft brick of the jupyter notebook, so beware")

    from os import listdir
    DataFolder = FileParentPath + "/0Data/"+ foldername + "/"
    filelist = [f for f in listdir(DataFolder)]
    print("The specified folder for display pictures and fractal derivatives: ",DataFolder)
    totaltime = 0.0
    counter = 0
    for index, filename in enumerate(filelist):
        counter +=1
        print("Picture",index+1,"of",whereTObreakIteration)
        if counter == whereTObreakIteration +1:
            break
        else:
            pass
        
        try:   
            filepath = DataFolder+ filename 
            print(filename)
            image = Image.open(filepath) #Load Image with Pillow
            
            #Cause rgb has 3 channels, the channels are flattend to 1 channel by concatenating the channels side by side
            ChannleDimension = len(str(image.mode)) # grey -> 1 chan , rgb 3 channle
            
            if verbosity == True:
                # summarize some details about the image
                print(image.format,image.size)
                print(image.mode)
                # show the image
                #image.show()
                print("Incoming image has",ChannleDimension, "channels")

            #init possible channels
            C1, C2, C3, C4, C5, C6 = None, None, None, None, None, None 
            channellist = [C1,C2,C3,C4,C5,C6]
            #Cropped channellist to actual used channels
            croppedChannelList = channellist[0:ChannleDimension-1]
            #slit channels to variables C1...C6
            croppedChannelList = image.split()        
            #Stacking channels from left to right
            initEntry = None
            stackedchannels = np.array(initEntry)
            for index, channel in enumerate(croppedChannelList):
                PicNumpy = np.array(channel)
                if verbosity == True:
                    channel.show()  
                    print(channel)
                    print(PicNumpy)
                    print(PicNumpy.shape)

                if index == 0:
                    stackedchannels = PicNumpy
                else:
                    stackedchannels = np.concatenate((stackedchannels,PicNumpy),axis=1)

            if verbosity == True:
                print(stackedchannels)
                print(stackedchannels.shape)
                
            npOutputFile = stackedchannels  #is the preprocessed picture as grayscale and flattened channels


            
            # MULTITHREAD BOXCOUNT Execution-------------------------------------

            start = time.time()   #start time here for just to compare boxcount performance, everything before is preprocessing 
                                  # and gpu-version also has preprocessing steps
            BoxCountR_SpacialLac_map_Dict = MultithreadBoxcount(npOutputFile)
        
            end = time.time()
            
            timethisround = end-start
            totaltime += timethisround

            BCRmap2 , Lakmap2 = BoxCountR_SpacialLac_map_Dict[0]
            BCRmap4 , Lakmap4 = BoxCountR_SpacialLac_map_Dict[1]
            BCRmap8, Lakmap8 = BoxCountR_SpacialLac_map_Dict[2]
            BCRmap16 , Lakmap16 = BoxCountR_SpacialLac_map_Dict[3]

            if display_images == "" or display_images == "y":

                showNPArrayAsImage(npOutputFile, "Original Picture", "gray")

                #source:[18] https://stackoverflow.com/questions/22053274/grid-of-images-in-matplotlib-with-no-padding

                max_cols = 2
                fig, axes = plt.subplots(nrows=4, ncols=max_cols, figsize=(10,16))
                lablelist = [BCRmap2, Lakmap2,BCRmap4 , Lakmap4,BCRmap8, Lakmap8,  BCRmap16 , Lakmap16 ]

                titlelist = ["BCR2", "LAC2", "BCR4", "LAC4", "BCR8", "LAC8","BCR16", "LAC16"]  #, "NN BCR2", "NN LAC2",   ]
                for idx, image in enumerate(lablelist):
                    row = idx // max_cols
                    col = idx % max_cols
                    #axes[row, col].axis("off")
                    axes[row,col].imshow(image, cmap="gray", aspect="auto")
                    axes[row, col].set_title(titlelist[idx])
                    # label x-axis and y-axis 
                    #axes[row, col].set_ylabel(ylabellist[idx]) 
                    #axes[row, col].set_xlabel(ylabellist[idx]) 

                plt.subplots_adjust(wspace=.1, hspace=.2)
            
            if foldername == "MISC":
                if savepicture == None:
                    savepicture = input("do you want to save pictures in the folder MISC into generated images? \n (Y/y) or not (n) or NEVER for just diplay output (N)")

                if savepicture == None or savepicture == "y" or savepicture == "Y" or savepicture == "":
                    saveplace = FileParentPath + "/0Data/generated_imgs/"+filename[:-3] +"png"
                    print("saveplace is", saveplace)
                    plt.savefig(saveplace , bbox_inches='tight')
                    continue

                elif savepicture == "N":
                    pass

                else:
                    savepicture = None
                    pass


            plt.show()
                            
            '''
            #Different kind of displaying images

            showNPArrayAsImage(BCRmap2, "BCRmap2", "gray")
            showNPArrayAsImage(Lakmap2, "Lakmap2", "gray")

            showNPArrayAsImage(BCRmap4, "BCRmap4", "gray")
            showNPArrayAsImage(Lakmap4, "Lakmap4", "gray")

            showNPAr21rayAsImage(BCRmap8, "BCRmap8", "gray")
            showNPArrayAsImage(Lakmap8, "Lakmap8", "gray")

            showNPArrayAsImage(BCRmap16, "BCRmap16", "gray")
            showNPArrayAsImage(Lakmap16, "Lakmap16", "gray")
            input()
            '''
            
        except :
            PrintException()
            print("Exception hit while showing pictures, assume abort, so break loop and continue with the script")
            input()
            break    
    
    #CALC time metrics to compare with gpu-Version
    mean_timePERbatch = totaltime / float(whereTObreakIteration)
    MegapixelPERsecond =  round( (1.0 * float(npOutputFile.shape[0]) * float(npOutputFile.shape[1])) /(mean_timePERbatch* 1000000 ) ,2)
    #print("compiler", compiler)
    print("CPU python code"," with", mean_timePERbatch, " seconds/Picture with a pixelthroughput of",MegapixelPERsecond )
    
youwanttoshow = input("Do you want to execute spacial boxcounting, track executed time and maybe display/save pictures? \n y/n")

if youwanttoshow == "y":
    Show_Pictures(foldername)

    
    

Do you want to execute spacial boxcounting, track executed time and maybe display/save pictures? 
 y/n n


## 3.4 Utilizing the capabilities of spacial Boxcounting by

The boxcount ratio is an indicator for spacial complexity like the fractal dimension, or hausdorff dimension and it can be calculated with boxcounts ranging over many scales. The more boxes out of all possible boxes are counted,  the more the boxcount ratio approaches one. Arrays with similar structures have  similar boxsize-scaled boxcount distributions, if the dimensions of the arrays remain constant.  
The lacunarity is an indicator for spacial heterogeneity on a specific scaling which is defined by the boxsize. The spacial heterogeneity can become useful for edgedetection on a specific scale, or dimensionallity reduction.

The two following tasks are just examples for using the boxcounting algorithm.

### 3.4.1 Similarity search

Comparing the boxcounts and lacunaritys of chosen boxsizes over many files can enable a controllable similarity search. 
The variable similarity equals the sum of the mean of all boxcount ratios for each boxsize and its diffrence to other files. The more the similarity is approaching zero, the more similar the chosen picture should be. 
To be accurate the spacial dimensions of pictures or arrays have to be equal in x,y and the maximum intrinsic value, when the lacunarity is used.
If just the boxcount ratio is used, also pictures/arrays of different sizes can be compared, because every boxcount is normalized against the maximum number of possible boxes for each boxsize respectively.

The similar picture search function is executed by using the folder name of the search space and the path to the picture, it should be compared to. The corresponding pictures are characterized with the spacial boxcounting and saved in a dictionary. Thereafter the similarity is calculated and sorted to display the first 50 images that are the most similar to the picture to be searched for. Since the default search picture is also in the folder of the search space, the second picture should be the same as the first one.


### 3.4.2  Edge detection with controllable edge width 

Edges often come with contrast, which results in an increased standard deviation in the value distribution of the input array cropped by the boxboundries.

These mathematical properties of the lacunarity and the spacial boxcounting derives a scaled down version of the pictures, which highlights regions of spacial complexity and heterogeneity. 
The lacunarity could be utilized in an edge detection algorithm highlighting all edges at a chosen scale, aka boxsize.

The example edge detection function is executed with an input picture to characterize it with the spacial boxcounting at all specified boxsizes. The results are displayed and the user can specify the scaling to choose the boxsize to get the desired output. Choosing a boxsize of 2 results in a second long task revealing fine edges in the picture. Increasing the boxsize will downscale the picture, which results in revealing coarser edges found within the picture and with much faster processing time in the millisecond range.





In [5]:
#Attention: Picture has to be same size/volume, when lacunarity is taken into account
#           when just using boxcounting ratio, pics can be different

#loaded picture to search for
pictureINquestion = FileParentPath +"/0Data/spacial_search_examples/"
#pictureINquestion += "hydrophobic_leaf_x1800.bmp"
#pictureINquestion += "7_3_1800x.bmp"
pictureINquestion += "Ripple_RM4_x1800.bmp"
#path to directory of search space
foldername = FileParentPath + "/0Data/Images/"

fractal_parameter_dict = {}    # dict with picturename and all scores for bcr/lak over boxsizes 2,4 8, 16

#simple example of search for similar pictures
def search_similar_picture_with(foldername, pictureINquestion):
    
    imageToSeachFor = Image.open(pictureINquestion)  # load picture in question

    #Cause rgb has 3 channels, the channels are flattend to 1 channel by concatenating the channels side by side
    ChannleDimension = len(str(imageToSeachFor.mode)) # grey -> 1 chan , rgb 3 channle

    if verbosity == True:
        # summarize some details about the image
        print(imageToSeachFor.format,imageToSeachFor.size)
        print(imageToSeachFor.mode)
        print("Incoming image has",ChannleDimension, "channels")

    #init possible channels
    C1, C2, C3, C4, C5, C6 = None, None, None, None, None, None 
    channellist = [C1,C2,C3,C4,C5,C6]
    #Cropp channellist to actual used channels
    croppedChannelList = channellist[0:ChannleDimension-1]

    croppedChannelList = imageToSeachFor.split()        
    #Stacking channels from left to right
    initEntry = None
    stackedchannels = np.array(initEntry)
    for index, channel in enumerate(croppedChannelList):
        PicNumpy = np.array(channel)
        if verbosity == True:
            channel.show()  
            print(channel)
            print(PicNumpy)
            print(PicNumpy.shape)

        if index == 0:
            stackedchannels = PicNumpy
        else:
            stackedchannels = np.concatenate((stackedchannels,PicNumpy),axis=1)

    if verbosity == True:
        print(stackedchannels)
        print(stackedchannels.shape)

    npOutputFileToSearchFor = stackedchannels

    
    # MULTITHREAD BOXCOUNT Execution----------------------
    # calc boxcountratio and lacunarity to characterize the picture
    start = time.time()   #start time here for just to compare boxcount performance, everything before is preprocessing 
    # and gpu-version also has preprocessing steps
    BoxCountR_SpacialLac_map_Dict = MultithreadBoxcount(npOutputFileToSearchFor)
    end = time.time()

    timethisround = end-start

    print(timethisround, "seconds for fractal characterization of the picture to be seached for.")
    
    BCRmap2 , Lakmap2 = BoxCountR_SpacialLac_map_Dict[0]
    BCRmap4 , Lakmap4 = BoxCountR_SpacialLac_map_Dict[1]
    BCRmap8, Lakmap8 = BoxCountR_SpacialLac_map_Dict[2]
    BCRmap16 , Lakmap16 = BoxCountR_SpacialLac_map_Dict[3]


    #take just the mean of the bcr ratios, since ratio is normalized against all possible boxes at each scaling
    # which lac is not and soforth maybe arrays of different sizes can be compared against
    meanBCR2 = np.mean(BCRmap2)
    meanBCR4 = np.mean(BCRmap4)
    meanBCR8 = np.mean(BCRmap8)
    meanBCR16 = np.mean(BCRmap16)

    
    # store bcr and lac for each scale in a dict with the key of the filname
    #fractal_parameter_dict[pictureINquestion] = np.array([meanBCR2,meanBCR4, meanBCR8, meanBCR16])

    SumLAC2 = np.mean(Lakmap2)
    SumLAC4 = np.mean(Lakmap4)
    SumLAC8 = np.mean(Lakmap8)
    SumLAC16 = np.mean(Lakmap16)
        
        # store bcr and lac for each scale in a dict with the key of the filname
    fractal_parameter_dict[pictureINquestion] = np.array([meanBCR2,meanBCR4, meanBCR8, meanBCR16, SumLAC2, SumLAC4, SumLAC8, SumLAC16])
    
    
    

    
    #CHARACTERIZING PICTURE TO BE SEARCHED FOR DONE,...
    #now iterate over the chosen folder and characterize as well

    #DataFolder = FileParentPath + "/0Data"+ foldername
    filelist = [f for f in listdir(foldername)]

    totaltime = 0.0
    counter = 0
    #iterate over all pictures and
    for index, filename in enumerate(filelist):
        counter +=1
        filepath = foldername+ filename 
        print(filename)
        #Load Image with Pillow
        image = Image.open(filepath)

        #Cause rgb has 3 channels, the channels are flattend to 1 channel by concatenating the channels side by side
        ChannleDimension = len(str(image.mode)) # grey -> 1 chan , rgb 3 channle

        if verbosity == True:
            # summarize some details about the image
            print(image.format,image.size)
            print(image.mode)
            # show the image
            #image.show()
            print("Incoming image has",ChannleDimension, "channels")

        #init possible channels
        C1, C2, C3, C4, C5, C6 = None, None, None, None, None, None 
        channellist = [C1,C2,C3,C4,C5,C6]
        #Cropp channellist to actual used channels
        croppedChannelList = channellist[0:ChannleDimension-1]
        #slit channels to variables C1...C6


        croppedChannelList = image.split()        
        #Stacking channels from left to right
        initEntry = None
        stackedchannels = np.array(initEntry)
        #stackedchannels = None
        for index, channel in enumerate(croppedChannelList):
            PicNumpy = np.array(channel)
            if verbosity == True:
                channel.show()  
                print(channel)
                print(PicNumpy)
                print(PicNumpy.shape)

            if index == 0:
                stackedchannels = PicNumpy
            else:
                stackedchannels = np.concatenate((stackedchannels,PicNumpy),axis=1)

        if verbosity == True:
            print(stackedchannels)
            print(stackedchannels.shape)

        npOutputFile = stackedchannels


        #calc bcr and lac

        # MULTITHREAD BOXCOUNT Execution

        start = time.time()   #start time here for just to compare boxcount performance, everything before is preprocessing 
                              # and gpu-version also has preprocessing steps
        BoxCountR_SpacialLac_map_Dict = MultithreadBoxcount(npOutputFile)
        end = time.time()

        timethisround = end-start


        BCRmap2 , Lakmap2 = BoxCountR_SpacialLac_map_Dict[0]
        BCRmap4 , Lakmap4 = BoxCountR_SpacialLac_map_Dict[1]
        BCRmap8, Lakmap8 = BoxCountR_SpacialLac_map_Dict[2]
        BCRmap16 , Lakmap16 = BoxCountR_SpacialLac_map_Dict[3]

        #TEST: take just the mean of the bcr ratios, since ratio is normalized against all possible boxes at each scaling
        # which lac is not and soforth maybe arrays of different sizes can be compared against
        meanBCR2 = np.mean(BCRmap2)
        meanBCR4 = np.mean(BCRmap4)
        meanBCR8 = np.mean(BCRmap8)
        meanBCR16 = np.mean(BCRmap16)



        SumLAC2 = np.mean(Lakmap2)
        SumLAC4 = np.mean(Lakmap4)
        SumLAC8 = np.mean(Lakmap8)
        SumLAC16 = np.mean(Lakmap16)
        
        # store bcr and lac for each scale in a dict with the key of the filname
        fractal_parameter_dict[filepath] = np.array([meanBCR2,meanBCR4, meanBCR8, meanBCR16, SumLAC2, SumLAC4, SumLAC8, SumLAC16])


    #sort the pictures with minimizing values scaled by scaledependent scaling factor

    #If you value coarse topograpfic features, just increase scalingfactor for the latter values
    #scalef. for Boxs.2, 4,    8,   16 
    #scaling for Lacunarity lower, cause bcr 0..1 and lac 0....xxxxxxx 
    scalinglist = np.array([0.25,0.25,0.25,0.25,0.05,0.05,0.05,0.05 ])  #scaling value for each boxsize aka scalingfactor

    similarity_dict = {}

    #tocompareagainst = np.array()
    tocompareagainst = fractal_parameter_dict[pictureINquestion]

    similarity_dict[pictureINquestion] = np.sum([0.0])

    for filepath, BCRlist in fractal_parameter_dict.items():
        if filepath == pictureINquestion:
            #if the searched for picture is current one, just continue to next element,
            # cause similarity is 0
            continue
        
        similarity = BCRlist - tocompareagainst   #Difference of current ratios against the values for the picture in question.
        similarity = np.absolute(similarity)  # we just care about tha absolut diffrence to the target
        similarity = np.multiply(similarity, scalinglist) #elementwise multiplication for scaling
        similarity = np.sum(similarity)   # to get to one value just take the sum of all elements
        
        similarity_dict[filepath] = similarity
    
    
    #source [19] https://stackabuse.com/how-to-sort-dictionary-by-value-in-python/
    fig=plt.figure()

    sorted_values = sorted(similarity_dict.values()) # Sort the values
    sorted_dict = {}

    for index, current_similarity in enumerate(sorted_values):
        if index == 0:
            print("Picture to search for")
        if index == 10:
            #just showing the first 50 pictures for conveniance
            break
            
        #loop for sorted values begin from 0 and ascending 
        for filepath in similarity_dict.keys():
            if similarity_dict[filepath] == current_similarity:
                #found entry
                # reading the image 
                foundImage = img.imread(filepath) 
                #img = PIL.Image.open(x)
                plt.subplot(4,4,index+1)
                chosentitle= "Similarity = ",str(current_similarity)
                plt.title(chosentitle)
                plt.axis('off')
                plt.imshow(foundImage, cmap='gray')

                #sorted_dict[key] = similarity_dict[key]
                break

        #print(sorted_dict)    
        plt.show(block=False)


youwanttosearch = input("Do you want to search for similar pictures in a dataset? \n y/n")

if youwanttosearch == "y":
    search_similar_picture_with(foldername, pictureINquestion)

    
    
#EDGEDETECTION--------------------------------
    
#pictureINquestion = FileParentPath +"/0Data/Images/"
#pictureINquestion += "7_4_1800x.bmp"
#pictureINquestion += "10_1_900x.bmp"
#pictureINquestion += "12_4_900x.bmp"

pictureINquestion =  FileParentPath +  "/0Data/spacial_search_examples/"
pictureINquestion += "edgedetect.bmp"


    
def edgedectecion_with(pictureINquestion):
    #load image
    ChosenImage = img.imread(pictureINquestion) 
    # full boxcounting on picture in question and view as a grid

    # MULTITHREAD BOXCOUNT Execution

    start = time.time()   #start time here for just to compare boxcount performance, everything before is preprocessing 
                          # and gpu-version also has preprocessing steps
    BoxCountR_SpacialLac_map_Dict = MultithreadBoxcount(ChosenImage)

    end = time.time()

    timethisround = end-start

    BCRmap2 , Lakmap2 = BoxCountR_SpacialLac_map_Dict[0]
    BCRmap4 , Lakmap4 = BoxCountR_SpacialLac_map_Dict[1]
    BCRmap8, Lakmap8 = BoxCountR_SpacialLac_map_Dict[2]
    BCRmap16 , Lakmap16 = BoxCountR_SpacialLac_map_Dict[3]
    
    showNPArrayAsImage(ChosenImage, "Original Picture", "gray")

    #source: [17] https://stackoverflow.com/questions/22053274/grid-of-images-in-matplotlib-with-no-padding

    max_cols = 2
    fig, axes = plt.subplots(nrows=2, ncols=max_cols, figsize=(14,10))
    lablelist = [ Lakmap2, Lakmap4, Lakmap8, Lakmap16 ]

    titlelist = ["LAC2", "LAC4", "LAC8","LAC16"]  
    for idx, image in enumerate(lablelist):
        row = idx // max_cols
        col = idx % max_cols
        #axes[row, col].axis("off")
        axes[row,col].imshow(image, cmap="gray", aspect="auto")
        axes[row, col].set_title(titlelist[idx])
        # label x-axis and y-axis 
        #axes[row, col].set_ylabel(ylabellist[idx]) 
        #axes[row, col].set_xlabel(ylabellist[idx]) 

    plt.subplots_adjust(wspace=.1, hspace=.2)
    plt.show()
    
    # evaluate the picture and what scaling and so forth boxsize for the edgedetection has to be choosed
    #choose boxsize
    print("0 detect fine grid(slow)... 3 detect with coarse grid (fast)")
    chosenscaling = int(input("Choose scaling ->[0,1,2,3] = [2,4,8,16] boxsize" ))
    start = time.time()   #start time here for just to compare boxcount performance, everything before is preprocessing 
    #chosenscaling = 0  #   s 0 = bz 2,    s 1 = bz 4,    s 2 = bz 8,   s 3 = bz 16
    #0 detects little edges  s3 just detects coarse edges
    maxvalue = 256 
    BoxCountR_SpacialLac_map = spacialBoxcount(ChosenImage, chosenscaling,maxvalue)
    end = time.time()

    timethisround = end-start
    
    print(timethisround," seconds passed for generating edgedetection map")
    BCRmap , Lakmap = BoxCountR_SpacialLac_map
    
    #set element-wise to True,  where element is over threshold
    #threshold = np.max(Lakmap)/2.0
    #Lakmap = Lakmap >= threshold
    showNPArrayAsImage(Lakmap, "Chosen Edgedetection Picture", "gray")

    #spacialboxcount
    # get lacmap 
    pass


youwanttoedgedetect = input("Do you want edgedetect the chosen picture? \n y/n")

if youwanttoedgedetect == "y":
    edgedectecion_with(pictureINquestion)

    

Do you want to search for similar pictures in a dataset? 
 y/n y


7.352 seconds for spacial boxcounting with  4 iterations/scalings
7.35248875617981 seconds for fractal characterization of the picture to be seached for.
10_1_1800x.bmp
2.154 seconds for spacial boxcounting with  4 iterations/scalings
10_1_3700x.bmp
1.981 seconds for spacial boxcounting with  4 iterations/scalings
10_1_900x.bmp
2.187 seconds for spacial boxcounting with  4 iterations/scalings
10_2_1800x.bmp
2.216 seconds for spacial boxcounting with  4 iterations/scalings
10_2_3700x.bmp
2.146 seconds for spacial boxcounting with  4 iterations/scalings
10_2_900x.bmp
2.387 seconds for spacial boxcounting with  4 iterations/scalings
10_3_1800x.bmp
2.086 seconds for spacial boxcounting with  4 iterations/scalings
10_3_3700x.bmp
1.959 seconds for spacial boxcounting with  4 iterations/scalings
10_3_900x.bmp
2.017 seconds for spacial boxcounting with  4 iterations/scalings
10_4_1800x.bmp
2.353 seconds for spacial boxcounting with  4 iterations/scalings
10_4_3700x.bmp
2.439 seconds for spacia

KeyboardInterrupt: 

## 3.5 Multi-label/-task learning with deep convolutional neural networks

Boxcounting like in paragraph 3.3.1 is a sequential task, which could be parallelized through utilizing a single convolutional neural network to calculate every resulting array specified in the multithreaded boxcounting function. The input of the convolutional neural network is a 2D array like the input picture in the cpu based boxcounting algorithm. The outputs of the network are the spacial boxcount ratios and lacunaritys to their respective scaling.

### 3.5.1 Problems and solutions

#### 3.5.1.1  Problem of different sized input shapes
Due to a technical formality all input arrays to the network need to be the same size.
To counter this problem, the incoming picture is going to be chunked into same sized arrays. Since common picture resolutions are mostly divisible by 2 and the chunks can be concatenated again after chunking, a reasonable picture size of 128x128 pixel was chosen.

 
#### 3.5.1.2 Helper functions :
The python script utilizes an argumentparser, which initiates an object containing all chosen options. To stay consistent with the code a option class object was initialized.
In case there are chunks, which have a different sized shape, the functions reshape_Data and pad check for different sizes and pad the missing values with zeros.
If you want to rebuild the training an testing datasets a helper function is needed to delete old datasets.
To automatically set the device to cpu or gpu for pytorch, the helper function get device is used.

In [ ]:
#Variables---------------------------------------------------
verbosity = False
ChunkLenght = 128  # The picture will be hacked into pieces of chosen Size -> more Datapoints, less Ram, ...

maxIndexX, maxIndexY = ChunkLenght, ChunkLenght
shape = (maxIndexX, maxIndexY )

#BOXCOUNT INIT
Boxsize=[2,4,8,16,32,64,128]    #,256,512,1024]
iteration = 0
#os.makedirs("images", exist_ok=True)

# Create a option object to store all variables
class OptionObject:
  def __init__(self, n_epochs, batch_size, img_size, channels, learning_rate, b1, b2 ):
    self.n_epochs = n_epochs
    self.batch_size = batch_size
    self.img_size = img_size
    self.lr = learning_rate
    self.b1 = b1   #first order momentum of gradient decay
    self.b2 = b2   #second order momentum of gradient decay
    self.channels = channels
    self.n_cpu = 0
    
#self, n_epochs, batch_size, img_size, channels, learning_rate, b1, b2
opt = OptionObject(100, 32, ChunkLenght, 1 , 0.00002, 0.5, 0.8)
img_shape = (opt.channels, opt.img_size, opt.img_size)



#Source: [20] https://stackoverflow.com/questions/35751306/python-how-to-pad-numpy-array-with-zeros
#@jit(nopython=False)  #,forceobj=True) # Set "nopython" mode for best performance, equivalent to @njit
def pad(array, reference, offset):
    """
    array: Array to be padded
    reference: Reference array with the desired shape
    offsets: list of offsets (number of elements must be equal to the dimension of the array)
    """
    # Create an array of zeros with the reference shape
    result = np.zeros(reference.shape)
    # Create a list of slices from offset to offset + shape in each dimension
    insertHere = [slice(offset[dim], offset[dim] + array.shape[dim]) for dim in range(array.ndim)]
    # Insert the array in the result at the specified offsets
    result[insertHere] = array
    return result


#If a picture/array is more little than the reference shape, than add zeros to the right and bottom with pad()-function to bring it into chunklenght x chunklenght
def reshape_Data(PicNumPy,shape, original_shape):
    #if shape is bigger then original, set new max and retry making data
    reshape = (int(original_shape[0]),int(original_shape[1]) )
    if verbosity:
        print("reshaping, cause shape and original shape are", shape, original_shape)
        print("reshape",reshape )
        print("PicNumPy shape",PicNumPy.shape)
    ### can add offset; so that pict can be centered
    offset = [0,0,0]
    PicNumPy = pad(PicNumPy,np.zeros(reshape),offset )
    return PicNumPy


#just functions via jupyter notebook with !command
def delete_dataset_from_last_time(FileParentPath):
    really = input("-->(y/n):  Do you want to delete the old Dataset? \n BE CARFUL: function can remove whole directorys, so dont change the Fileparentpath")
    
    
    if really =="y":
        OldDatasetSaveplace = FileParentPath+"/Datasets/test/"
        try:
            os.rmdir(OldDatasetSaveplace)
            os.mkdir(OldDatasetSaveplace)
            os.mkdir(OldDatasetSaveplace+"/features/")
            os.mkdir(OldDatasetSaveplace+"/labels/")
        except OSError:
            print("Deleting old test dataset failed")
            PrintException()
        else:
            print("Old test dataset deleted!")
        
        #!rm -rf OldDatasetSaveplace
        #!mkdir OldDatasetSaveplace
        #!mkdir OldDatasetSaveplace +"/features/"
        #!mkdir OldDatasetSaveplace +"/labels/"


        OldDatasetSaveplace = FileParentPath+"/Datasets/train/"
        try:
            os.rmdir(OldDatasetSaveplace)
            os.mkdir(OldDatasetSaveplace)
            os.mkdir(OldDatasetSaveplace+"/features/")
            os.mkdir(OldDatasetSaveplace+"/labels/")
        except OSError:
            print("Deleting old train dataset failed")        #!rm -rf OldDatasetSaveplace
            PrintException()
            input("Press any key to continue")
        else:
            print("Old test dataset deleted!")
        
        #!mkdir OldDatasetSaveplace
        #!mkdir OldDatasetSaveplace +"/features/"
        #!mkdir OldDatasetSaveplace +"/labels/"
    else:
        print("Continue without deleting the old dataset")
        input("Press any key to continue")


#Setting device to GPU/CPU 
def get_device():
    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'

    return device

device = get_device()
print("Chosen Devide is",device)







### 3.5.2  Data preprosessing


#### 3.5.2.1 Increased variance balancing
In a balanced dataset the number of each class should be about equal to prevent the network from overfitting. This is accomplished by under- and oversampling, so to discard overrepresentative data or duplicate underrepresentative data. Since there are no prelabeled classes to this data and the output is matrix wise and quasicontinuous, balancing this data is based on the assumptions, that the fractal parameters are a quantitative measurement for the spacial dimensionality of an object and by that balancing can be achieved.
The dataset is initiated with a base population of pictures and returns the sumed boxcount ratio and lacunarity for a chosen boxsize. A variance is calculated and by adding a new element to the dataset, the variance of the chosen values is in- or decreasing. With increasing variance, it is assumed, the new element is something unseen and increases the variance of the dataset. If the variance decreases, it is assumed, similar content already exists in the dataset and copies or slight alterations will more likely tend the neural network to overfit.


To have some sort of controlling the aggressiveness of the train and test splitting, the variable precision is used to round the old and new variance. If the new variance is the same of bigger, add the current picture to the training dataset.If the variance is lower, add the current picture to the testing dataset. When the variable precision is zero, don't balance and just create a training dataset.


#### 3.5.2.2 Make and split train and test data

To be able to generate a dataset for machine learning purposes, some data preprocessing steps have to be executed, to learn the desired patterns accordingly.

The dataset generation process iterates over the files of the specified path and loads the pictures as an array.
To address pictures with multiple channels, like rgb, the channels are flattend by concatenating the arrays side by side from left to right.
Due to a technical formality, the tensors in the machine learning model have to be consistent in size, so the loaded images are chunked into fixed grid areas with the size of 128² pixel.
Neuronal networks can just work in the range of their activation function, so all features (images) and labels (spacial boxcounting results) have to be normalized from -1 to 1.

The features and labels are indexed and saved in the subfolder named datasets, according to the train, test dataset split.





In [ ]:
#   Data Balancing/Reshaping Part-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
'''
The data has to be balanced in multiple ways to prevent overfitting.
For example not all pictures can be used to train data. 
Binary classes can be balanced just by taking 50/50 balance for the training dataset.
Cause the lables are the calculated arrays of a cpu driven program, 
there is just a continuum of output arrays for input arrays.
'''
verbosity = False
precision = 1   # 0 dont balance, 1 balance light, ...9 balance fine; The finer, the more data will be discarded


#if returns True, dataset stays balanced, so take into train-data, else pack into test set. 
#@jit(nopython=False)  #,forceobj=True) # Set "nopython" mode for best performance, equivalent to @njit
def CalcPlacingcondition(DensityMap, sumBCR,sumLAK, precision, lastVariance, index):
    #print("DensityMap",DensityMap)
    print("DensityMap.shape",DensityMap.shape)

    element = np.array([[sumBCR,sumLAK],])
    
    if index ==0:
        combinedDensityMap = element     #init first element
    else:
        #concatenate the BCR and LAK from the current element to the 
        combinedDensityMap = np.concatenate((DensityMap,element),axis=0)

    
    if precision == 0:
        placingcondition = True
        DensityMap = combinedDensityMap
        lastVariance = np.var(combinedDensityMap)
    else:
        if index <= 6:
            #to populate the field, just add the first 6 elements
            placingcondition = True

            #cause element placingcondition is true, update the variances             
            combinedVariance = np.var(combinedDensityMap)
            lastVariance =  combinedVariance
            print("populate with minimum Pop")
        else:
            #calculate the Variance from this turn
            combinedVariance = np.var(combinedDensityMap)

            #if the rounded variance of this turn is more or the same of the variance last turn
            if round(combinedVariance,precision) >= round(lastVariance,precision):    
                placingcondition = True
                #and update the last variance for next turn with the new value
                lastVariance =  combinedVariance
                DensityMap = combinedDensityMap
            else:
                #dont add this element to the training dataset but to the test dataset
                placingcondition = False

            if verbosity: print("index:", index,"    formerVariance,combinedVariance",round(lastVariance,precision),round(combinedVariance,precision),"so placing is",placingcondition)    
            #input()

    return placingcondition, DensityMap, lastVariance



def make_train_test_data(shape):
    
    Num_test = 0
    Num_train = 0
    firsttime = True

    DensityMap= np.array([[0.0,0.0],])
    lastVariance = 0.0
    original_shape = shape

    #For Visualizing  tqdm-progress bar
    from tqdm import tqdm
    pbar = tqdm(total=maxIndexY-1)
    counter = 0
    train_counter = 0
    test_counter =0

    from os import listdir
    DataFolder = FileParentPath + "/0Data/Images/"
    filelist = [f for f in listdir(DataFolder)]
    #print(filelist)



    from tqdm import tqdm
    pbar = tqdm(total=len(filelist))

    
    n = 4 # werden wohl bis max 16 begrenzen # ist 4 da beim slicen  richitg wäre  boxsize[:4] oder [:-7]   

    for index, filename in enumerate(filelist):
        pbar.update(1)  #updates progressbar
        counter +=1
        try:   
            filepath = DataFolder+ filename  #Load Image with Pillow
            # Open the image
            image = Image.open(filepath)
            
            # If the picture is in RGB or other multi-channel mode 
            #just seperate the channels and concatenate them from left to right
            ChannleDimension = len(str(image.mode)) # grey -> 1 chan , rgb 3 channle
            
            if verbosity == True:
                # summarize some details about the image
                print(image.format,image.size)
                print(image.mode)
                #show the image
                image.show()
                print("ChannleDimension",ChannleDimension)

            channelcodierung = []
            for channel in image.mode:
                #FLATTEN EACH CHANNEL TO ONE  BY TILLING, cause cnn have to be consistent channles
                #and if one rgb is in grayscale, then error
                if verbosity: print(channel)
                channelcodierung.append(channel)
            
            C1, C2, C3, C4, C5, C6 = None, None, None, None, None, None
            channellist = [C1,C2,C3,C4,C5,C6]
            croppedChannelList = channellist[0:ChannleDimension-1]
            croppedChannelList = image.split()        
            initEntry = None
            stackedchannels = np.array(initEntry)
            for idx, channel in enumerate(croppedChannelList):
                PicNumpy = np.array(channel)
                if verbosity == True:
                    print(PicNumpy)
                    print(PicNumpy.shape)

                if idx == 0:
                    stackedchannels = PicNumpy
                else:
                    stackedchannels = np.concatenate((stackedchannels,PicNumpy),axis=1)
                
            
            if verbosity: print(stackedchannels.shape)
            npOutputFile = stackedchannels

            
            #########################################################################
            # MULTITHREAD BOXCOUNT LABLE EXTRACTION
            BoxCountR_SpacialLac_map_Dict = MultithreadBoxcount(npOutputFile)

            
            ##### CALC PLACING CONDITION FOR EACH DATAPOINT(PICTURE)
            maxiteration = 3 # cause boxsize 2,4,8,16 = 0,1,2,3 iteration # determines the maximum boxsize
            #breaking it down to 2 values
            sumBCR, sumLAK = BoxCountR_SpacialLac_map_Dict[1][0], BoxCountR_SpacialLac_map_Dict[1][1]             #BoxCountR_SpacialLac_map_Dict[4] #[0], BoxCountR_SpacialLac_map_Dict[4][1] 
            if verbosity: print("BCR,sumLAK  MAP before sum", sumBCR,sumLAK)

            sumBCR, sumLAK = np.sum(sumBCR), np.sum(sumLAK)
            if verbosity: print("sumBCR,sumLAK after sum", sumBCR,sumLAK)
            
            #Calc, if the next data in the dataset will balance it more or not
            placingcondition, DensityMap, lastVariance = CalcPlacingcondition(DensityMap, sumBCR,sumLAK,precision,lastVariance, index)

            maxChunkCount = (npOutputFile.shape[1]/ChunkLenght) * (npOutputFile.shape[0]/ChunkLenght)   #Chunks the picture is broken down into
            #print("maxChunkCount", maxChunkCount)
            Chunks = [None] * int(np.ceil(maxChunkCount))     # round up the non full boxes, cause they will be reshaped by padding with zeros       
            start = time.time()

            BoxBoundriesY = [0,ChunkLenght]
            BoxBoundriesX = [0,ChunkLenght]

            iteration = 0       # is Boxsize 32 -> should be faster then more little boxsize
            Boxsize=[2,4,8,16,32,64,128,256,512,1024]
            scalingFaktor = 1.0 / float(Boxsize[iteration])
            #If we take a sclice from the BCRmap/LAKmap, the boxboundries have to  be scaled for maintaining spacial dimensions across scaling with iteration and Boxsize
            Scaled_BoxBoundriesY = [0,int(ChunkLenght*scalingFaktor)]
            Scaled_BoxBoundriesX = [0,int(ChunkLenght*scalingFaktor)]


            for i in range(len(Chunks)):

                Chunks[i] = npOutputFile[BoxBoundriesY[0]:BoxBoundriesY[1],BoxBoundriesX[0]:BoxBoundriesX[1]] 

                CHUNKED_BoxCountR_SpacialLac_map_Dict = {}
                CuttedBoxsizeList = Boxsize[:maxiteration+1]
                if verbosity == True:
                    print("Current box",i,"of all",len(Chunks),"boxes")
                    print("Boxboundries: X, Y :" ,BoxBoundriesX,BoxBoundriesY)
                    print("CuttedBoxsizeList", CuttedBoxsizeList)
                    print("Converting BCRmap,LAKmap into Chunked Form")
                    
                for it , currentboxsize in enumerate(CuttedBoxsizeList):
                    if verbosity: print("Iteration", it, " and currentBoxsize", currentboxsize)
                    
                    scalingFaktor = 1.0 / float(currentboxsize)
                    
                    try:
                        #calc Scaled BoxBoundriesY
                        Scaled_BoxBoundriesY = [int(BoxBoundriesY[0]*scalingFaktor),int(BoxBoundriesY[1]*scalingFaktor)]

                    except:
                        PrintException()
                        #Assuming devide by zero
                        Scaled_BoxBoundriesY = [0,int(BoxBoundriesY[1]*scalingFaktor)]

                    try:
                        #calc Scaled BoxBoundriesX
                        Scaled_BoxBoundriesX = [int(BoxBoundriesX[0]*scalingFaktor),int(BoxBoundriesX[1]*scalingFaktor)]

                    except:
                        PrintException()
                        #Assuming devide by zero
                        Scaled_BoxBoundriesX = [0,int(BoxBoundriesX[1]*scalingFaktor)]                       


                    BCRmap, LAKmap = BoxCountR_SpacialLac_map_Dict[it]
                    
                    chunked_BCRmap = BCRmap[Scaled_BoxBoundriesY[0]:Scaled_BoxBoundriesY[1],Scaled_BoxBoundriesX[0]:Scaled_BoxBoundriesX[1]] 
                    chunked_LAKmap = LAKmap[Scaled_BoxBoundriesY[0]:Scaled_BoxBoundriesY[1],Scaled_BoxBoundriesX[0]:Scaled_BoxBoundriesX[1]] 
                    
                    #SCALING----------------------------
                    #Scale the values of the Arrays in a gaussian distrubution with mean 0 and diviation 1?!?!?!
                    #chunked_BCRmap, chunked_LAKmap = preprocessing.scale(chunked_BCRmap) , preprocessing.scale(chunked_LAKmap) 

                    '''
                    ATTENTION SCALING DEACTIVATED
                    '''
                    
                    #Normalize the Values betweeen -1...1----------------------------
                    chunked_BCRmap, chunked_LAKmap = preprocessing.normalize(chunked_BCRmap, norm='l1')  , preprocessing.normalize(chunked_LAKmap, norm='l1')  


                    if verbosity == True:
                        showNPArrayAsImage(Chunks[i], "Current Chunk", "gray")
                        print("chunked_BCRmap",chunked_BCRmap)
                        print("chunked_LAKmap",chunked_LAKmap)
                        showNPArrayAsImage(chunked_BCRmap, "chunked_BCRmap", "gray")
                        showNPArrayAsImage(chunked_LAKmap, "LAKmap", "gray")                    
                    
                    #index the BCR /LAK map to the right size 
                    CHUNKED_BoxCountR_SpacialLac_map_Dict[it] = [chunked_BCRmap, chunked_LAKmap]


                #Scale Dataset : Scaled data has zero mean and unit variance
                #Chunks[i]= preprocessing.scale(Chunks[i])
                
                '''
                ATTENTION SCALING DEACTIVATED
                '''
                
                
                #Normalizing ARRAY  from 0...255 to -1...+1
                Chunks[i] = preprocessing.normalize(Chunks[i], norm='l1')        
                Chunkshape = Chunks[i].shape

                if BoxBoundriesX[1] > npOutputFile.shape[1]  or BoxBoundriesY[1] > npOutputFile.shape[0]:
                    if verbosity: print("Chunkshape and shape are Different... reshaping")
                    Chunks[i] =  reshape_Data(Chunks[i], Chunkshape, shape)
                    continue

                assert Chunks[i].shape == original_shape

                newshape = ( 1,int(original_shape[0]),int(original_shape[1]) )
                Chunks[i] = np.reshape(Chunks[i], newshape)

                #saving test or Train image and label
                feature =  Chunks[i]
                label =  [np.array(CHUNKED_BoxCountR_SpacialLac_map_Dict[0]) , np.array(CHUNKED_BoxCountR_SpacialLac_map_Dict[1]) , np.array(CHUNKED_BoxCountR_SpacialLac_map_Dict[2]) , np.array(CHUNKED_BoxCountR_SpacialLac_map_Dict[3]) ]
                
                if verbosity == True:
                    print("feature",feature)
                    print("label",label)

                if placingcondition == True:
                    if verbosity: print("Placingcondition is true, Append Chunk to dataset")
                    #saveplace
                    trainsaveplace = FileParentPath+"/Datasets/train/"
                    #saving image
                    imagesaveplace = trainsaveplace+ "/features/"+"Feature"+ str(Num_train)
                    np.save(imagesaveplace, feature)
                    #saving label
                    labelsaveplace = trainsaveplace+ "/labels/"+"label"+ str(Num_train)
                    #CANT save List with different sized np arrays with np.save -> use pickle as workaround
                    pickle.dump(label,open(labelsaveplace,"wb"))
                    Num_train +=1
                        
                else:
                    testsaveplace = FileParentPath+"/Datasets/test/"
                    #saving image
                    imagesaveplace = testsaveplace+ "/features/"+"Feature"+ str(Num_test)
                    np.save(imagesaveplace, feature)
                    #saving label
                    labelsaveplace = testsaveplace+ "/labels/"+"label"+ str(Num_test)
                    #np.save(labelsaveplace, label)
                    #CANT save List with different sized np arrays with np.save -> use pickle as workaround
                    pickle.dump(label,open(labelsaveplace,"wb"))
                    Num_test +=1



                #After this Chunk set the new Borders of the new chunk for next turn

                if BoxBoundriesX[1] < npOutputFile.shape[1]:

                    BoxBoundriesX[0] =BoxBoundriesX[0] + maxIndexX
                    BoxBoundriesX[1] = BoxBoundriesX[1] + maxIndexX
                    if verbosity == True: 
                        print("Move box into x direction")
                        print("BoxBoundriesX", BoxBoundriesX)
                else:
                    if verbosity == True:
                        print(BoxBoundriesY,"BoxBoundriesY") 
                        print("move box into starting position in x-direction")
                        print("move box into ydirection")
                    BoxBoundriesX[0]=0
                    BoxBoundriesX[1]=maxIndexX
                    BoxBoundriesY[0]+=maxIndexY
                    BoxBoundriesY[1]+=maxIndexY
                    
            if verbosity: input("Press any key for next File")


            end = time.time()     
            print(round(end,1) - round(start,1), "seconds passed for chunking and Make Train Data for 1 File")

        except :
            PrintException()
            input()
            pass
    
    pbar.close()        #close the percentage bar cause all pictures have been processed

    if verbosity:
        #To evaluate, if balancing happened  show figure of all BCR/LAKS of the training dataset
        x,y = np.array([]) , np.array([])
        for Koordinate in DensityMap:
            x,y = np.append(x,Koordinate[0]) , np.append(y,Koordinate[1])

        # Plot
        plt.scatter(x, y,s=1, alpha=0.33)
        plt.title('Lacunarity-Boxcountratio Diagramm')
        plt.xlabel('sumBCR')
        plt.ylabel('MeanLAK')
        plt.show()

    print("Num_train",Num_train ,"Num_test",Num_test)



    

In [ ]:

#If rebuild  data == True, take the chosen DataFOLDER and create a train and test Dataset 
youwanttorebuild = input("y/N   Do you want to rebuild the machine learning dataset? \n Just has to be executed once.")

if youwanttorebuild == "y":
    REBUILD_DATA = True
elif youwanttorebuild == "" or youwanttorebuild == "n" or youwanttorebuild == "N":
    REBUILD_DATA = False  # set to true to run once, then back to false unless you want to change something in your training data.

    
if REBUILD_DATA == True:
    delete_dataset_from_last_time(FileParentPath)
    make_train_test_data(shape)
    
#REBUILDING/Balancing DATA DONE ---------------------------------------------------------------------------------------------

    

### 3.5.3 Define custom multi label dataset for pytorch

The multi label approach needs to define a custom dataset, using the pytorch dataloader. It automatically handels multiprocessing to keep hardware utilization high during training. The training and testing dataset is generated from the files, which are saved by the data preprocessing function (3.5.5.2) in the subfolder datasets.

In [ ]:
# Create COUSTOM Pytorch DATASET with features and labels----------------------------------------------------------------------------
# Source: [21] https://stackoverflow.com/questions/56774582/adding-custom-labels-to-pytorch-dataloader-dataset-does-not-work-for-custom-data
try:
    import platform
    Operating_system = platform.system()
    print(Operating_system)
    if Operating_system == 'Windows':
        print("Windows detected, dataloader multiprocessing not avaiable. Set n_cpu to 0 ")
        opt.n_cpu = 0
    from os import listdir
    from os.path import isfile, join

    class Dataset:
        def __init__(self, root):
            """Init function should not do any heavy lifting, but
                must initialize how many items are availabel in this data set.
            """
            self.featurepath = root + "/features"
            self.labelpath = root + "/labels"

            self.ROOT = root
            self.featurelist = [f for f in listdir(self.featurepath) if isfile(join(self.featurepath, f))]
            self.labellist = [f for f in listdir(self.labelpath) if isfile(join(self.labelpath, f))]

        def __len__(self):
            """return number of points in our dataset"""
            return len(self.featurelist)

        def __getitem__(self, idx):
            """ Here we have to return the item requested by `idx`
                The PyTorch DataLoader class will use this method to make an iterable for
                our training or validation loop.
            """
            imagepath =   self.featurepath+"/"+ "Feature" +str(idx)+".npy"
            img = np.load(imagepath)
            labelpath =   self.labelpath+"/"+ "label"+str(idx)
            #Below is to read and retrieve its contents, rb-read binary
            with open(labelpath, "rb") as f:
                label = pickle.load(f) 
                labels_2 = np.array(label[0])
                labels_4 = np.array(label[1])
                labels_8 = np.array(label[2])
                labels_16 = np.array(label[3])
            return img, labels_2 , labels_4 , labels_8, labels_16

    #And now, you can create an instance of this class as,
    trainDatasetSaveplace = FileParentPath + "/Datasets/train"
    trainDataset = Dataset(trainDatasetSaveplace)
    #Now, you can instantiate the DataLoader:
    trainDataloader = DataLoader(trainDataset, batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu, drop_last=True)
    dataiter = iter(trainDataloader)
    trainDataset = dataiter.next()
    trainDataset  = transforms.ToTensor()

    #DEFINING TEST DATA LOADER FOR TESTINGs

    testDatasetSaveplace = FileParentPath + "/Datasets/test"
    testDataset = Dataset(testDatasetSaveplace)
    testDataLoader = DataLoader(testDataset, batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu, drop_last=True )
    #This will create batches of your data that you can access as:
    testiter=  iter(testDataLoader)
    testDataset = testiter.next()
    testDataset  = transforms.ToTensor()


    #####################################################################################################################
    #       DATA COMPLETE       ->     NOW MACHINE LEARNING PART

except:
    PrintException()
    print("Did you rebuild the train test data? Please check")
    pass




### 3.5.4 Defining network architecture

While the cpu based spacial boxcounting algorithm has to be executed for every boxsize in a different thread, there may be many similar or equal computation steps between threads, which could be saved.
A single input multi-channel/-label output machine learning model to compute everything in one forward pass is declared.
The class "BoxcountEncoder" is defined by a integer-based layer discription, which is passed to it at the initialization step to generate a convolutional neural network.
The network gets a batch of arrays as input. 
At each convolutional layer the picture is scaled down by the factor of 2, to reduce the size in x- and y-direction with the help of a convolutional layer with the kernel size and a stride of 2x2.
The network receives one input channel due to the grayscale 8-bit image. The channels of the hidden layers can be chosen arbitrarily and are optimizable hyperparameters. A schematic of this neural network architecture is shown in figure 9.

Because the hidden layers have way more than 2 channels, each layer and so forth downscaling an output branch is added. This output branch utilizes a 1x1 conv layer with two output channels, to compare the network output against the spacial boxcounting ratio and lacunarity arrays in a pixelwise lossfunction.
The internal activation functions are composed of leaky rectified linear units, named relu for fast convergence.
The output layers are activated by a hyperbolic tangent function, which allows for scaled input data with mean of zero and standard deviation of one. Due to the fact that scaling changes the data distribution within the array and we just want to quantitatively characterize the containing original data, it was disabled, but the hyperbolic tangent function is still in place.
If scaling is disabled the sigmoid activation function is another option to apply to the output layers ranging from 0 to 1, because 8-bit images are only positive bits ranging from 0 to 255.
The layer discripton also allows batch-normalization layers, which effects were not used or tested yet.



![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/Document_images/networkarch2.png)

Figure 9: Multi label output model architecture.   
Source: http://alexlenail.me/NN-SVG/AlexNet.html


In [ ]:
verbosity = False
#INIT BoxcountNetParams as empty Dict
BoxCountParameter = {}

#The BoxCountEncoder takes the image and calculates the boxcountratios and the Lakcountmaps for each iteration/boxsize/scale and returns it.

class BoxCountEncoder(nn.Module):
    def __init__(self,Parameter):
        super(BoxCountEncoder, self).__init__()
        #Boxcount EncoderLayer<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        self.LayerDiscription = Parameter['BoxCountLayerDiscription']   #
        self.input_shape = Parameter['input_shape']
        self.LayerCount = len(self.LayerDiscription)
        self.Layers = nn.ModuleList() 
        self.OutputLayerIndexList = Parameter['OutputLayerIndexList']
        if verbosity == True:
            print("self.OutputLayerIndexList", self.OutputLayerIndexList)

        #Throughput=[]   #list of layercount for Batchnormlayer
        #ATTENTION  ENHANCER BUILDS NETWORK BACKWARDS AND IN AND OUT ARE ALSO SWITCHED
        for i in range(self.LayerCount):        #iterate forwards
            IN,OUT,Kx, Ky,Sx,Sy,Px,Py,BN = self.LayerDiscription[i]
            if verbosity: print("Layer",i," with Parameters", IN,OUT,Kx, Ky,Sx,Sy,Px,Py,BN)
            self.Layers.append(nn.Conv2d(IN,OUT, kernel_size=(Kx, Ky), stride=(Sx, Sy), padding=(Px, Py)) )      #Attention Compressor INOUT NOrmal
            
            #if this is an output layer, then use tanH instead of relu to prevent jumps for values around zero
            # tanh also allows scaled data, which it's mean is around zero and standard devitation of one
            if OUT==2 and  Kx==1 and Ky==1 and Sx==1 and Sy ==1 or  int(i) == int(self.LayerCount)-1: 
                if verbosity: print("Output Layer found")
                #self.Layers.append(nn.Sigmoid())
                self.Layers.append(nn.Tanh())
                #Cause tanh is another layer it has to be in  self.OutputLayerIndexList, so it dosent connect to x but branched out to y
            else:
                #Just use LeakyReLU for fast convergence
                self.Layers.append(nn.LeakyReLU(inplace = True))
   
            if BN ==1:
                self.Layers.append( nn.BatchNorm2d(OUT, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) )      #Attention Compressor INOUT NOrmal
                #Throughput.append(OUT)      

        if verbosity: print("self.input_shape",self.input_shape)
        print(self.Layers)
        print("-------------INIT DONE ------------------ \n ")



    def forward(self, x):

        output2, output4, output8, output16 = None, None, None, None
        OutputList = [output2, output4, output8, output16]
        outputindex = 0
        for i, layer in enumerate(self.Layers):        #iterate forwards
            #IN,OUT, Kx, Ky,Sx,Sy,Px,Py,BN = self.LayerDiscription[i]

            if i in self.OutputLayerIndexList:      #If this is a Output layer
                out = self.Layers[i](x)     # create branch and dont overwrite x
                #prints commented out, cause foreward will be executed millions of times
                #print("Create Branch")
            elif i-1 in self.OutputLayerIndexList:      #If this is a Output layer ACTIVATION FUNCTION (tanh)
                out = self.Layers[i](out)     # ACtivate out with act. fct
                OutputList[outputindex] = out       # set value for each output-layer / scale 
                #print("Activate Branch for output",outputindex)                
                outputindex +=1 
            else:
                x = self.Layers[i](x)
                #print("Append layer to Main Branch")

        return OutputList[0] , OutputList[1] , OutputList[2] , OutputList[3]




### 3.5.6 Define training and saving strategy for the convolutional neural network model

Training convolutional neural networks consumes a lot of computational power and time to converge against zero in a given loss function. There also are many hyperparameters and a model saving strategy to choose, because things might go wrong in many ways, so saving the progress occasionally is a important part, when the training can take several days of computation.

To tackle the hyperparameter optimization, the python library hyperopt was applied to the training part in form of the following training function. The training function is given a hyperparameter-space dictionary and the boxcount encoder class to generate a neural network. The loss function is defined by the pixelwise loss between the net generated output and the real cpu-calculated labels. The lossfunction of each boxsize is multiplied by a chosen fraction to gain control for weighing each boxsize. The weighed lossfunctions are sumed up to the general lossfunction and backpropagation is applied over the complete model. The chosen optimizer is adam with the learning rate, the first and second moment of learning rate decay.

The saving of new models will be executed, if the loss function of the current model is lower than the previous loss function. At this point the model has to be saved in 2 files, containing the network parameters, which are able to generate the chosen model architecture and the models weights and biases in form of a state dictionary. 

In [ ]:
previous_Best_Loss = None
Loss_Now = None

def TrainSpacialBoxcount_with(HyperparameterAndENCODERCLASS):
    HyperParameterspace, BoxCountEncoder =  HyperparameterAndENCODERCLASS 

    global previous_Best_Loss, Loss_Now
    #BoxCountEncoder = None

    #INIT VARIABLES
    opt.n_epochs = HyperParameterspace['n_epochs']
    opt.batch_size = HyperParameterspace['batch_size']
    opt.lr = HyperParameterspace['lr']
    opt.b1 = HyperParameterspace['b1']
    opt.b2 = HyperParameterspace['b2']

    #BoxCountLayerDiscription
    #self.BoxcountRatioConv = nn.Conv2d(3, 16, (self.BoxsizeX,BoxsizeY), (len(self.BoxsizeX), len(self.BoxsizeY) ), padding=0) 
    #self.LACcountConv = nn.Conv2d(3, 16, (self.BoxsizeX,self.BoxsizeY), (len(self.BoxsizeX), len(self.BoxsizeY) ), padding=0) 
    Boxsize=[2,4,8,16,32,64,128,256,512,1024]

    #Channles
    IN, OUTCOM = 1 , 2      #Channels 2, cause output is BCRmap and LAKmap, one is derived from the other
    Inter1, Inter2, Inter3    = HyperParameterspace['Inter1'], HyperParameterspace['Inter2'], HyperParameterspace['Inter3']          # Hyperoptimization here
    #Kernelsize in X/Y resprective layer is 2, cause...
    Kx1, Kx2, Kx3, Kx4              = 2,2,2,2       
    Ky1, Ky2, Ky3, Ky4              = 2,2,2,2
    # ...with a stride of 2 the picture is getting halfed in size, exacly like the boxcounting with bigger boxsizes , but in cpu version the ori bz overlap one pixel, which here is not checkk
    Sx1, Sx2, Sx3, Sx4              = 2, 2, 2, 2
    Sy1, Sy2, Sy3, Sy4              = 2, 2, 2, 2
    #padding should be 0, cause every picture is same size and all the kernels fit perfectly
    Px1, Px2, Px3, Px4              = 0,0,0,0
    Py1, Py2, Py3, Py4              = 0,0,0,0
    #Batchnorm is not needed, causewe want to focus just on the convolution calculation  and dont want to alter the image/ entry arrays in any unknown form..
    BN1, BN2, BN3, BN4              = 0,0,0,0

    #Intermediate OutputLayer ... Cause every Filter of a Convulution  is described within the Channels in the hidden layers and we want to output the BCR and LAK like like the cpu version...
    # we have to generate an output with a 1x1 = KxS conv with x Chan input and 2 Chan Output for calcing the loss

    BoxCountLayerDiscription = [    [IN,    Inter1,Kx1, Ky1,Sx1,Sy1,Px1,Py1,BN1],   #input layer
                                    [Inter1,OUTCOM ,1, 1,1,1,0,0,0],                # output layer for first iteration (Boxsize 2)

                                    [Inter1,Inter2,Kx2, Ky2,Sx2,Sy2,Px2,Py2,BN2],
                                    [Inter2,OUTCOM ,1, 1,1,1,0,0,0],                # output layer for second iteration (Boxsize 4)
        
                                    [Inter2,Inter3,Kx3, Ky3,Sx3,Sy3,Px3,Py3,BN3],
                                    [Inter3,OUTCOM ,1, 1,1,1,0,0,0],                # output layer for third iteration (Boxsize 8)

                                    [Inter3,OUTCOM,Kx4, Ky4,Sx4,Sy4,Px4,Py4,BN4],   #last ouput layer
                                            ]                       # [Inter3,OUTCOM ,1, 1,1,1,0,0,0],                # output layer for 4th iteration (Boxsize 16)

    input_shape = (opt.batch_size,1, ChunkLenght,ChunkLenght)

    OutputLayerIndexList = [2,6,10,12]        # Cause the intermediate output layers are branched out from the main flowchart

    BoxCountNetParameters = {'BoxCountLayerDiscription': BoxCountLayerDiscription, 'input_shape': input_shape, 'OutputLayerIndexList': OutputLayerIndexList}


    Modelname =  "n_epochs_" + str(round(opt.n_epochs,3)) 
    Modelname += "_batch-size_" + str(round(opt.batch_size,3))   
    Modelname += "_learning-rate_" + str(round(opt.lr,3))
    Modelname += "_beta-decay_" + str(round(opt.b1,3)) +"_" + str(round(opt.b2,3))
    #Modelname += "_Scalefactors_" + str(round(Scalefactor_2,3)) +"_" + str(round(Scalefactor_4,3)) +"_" + str(round(Scalefactor_8,3))

    # -----------------
    #  Train_BoxcountingCONV
    # -----------------
    #define Loss
    pixelwise_loss = torch.nn.L1Loss()

    #Init BoxcountEncoder
    BoxCountEncoder = BoxCountEncoder(BoxCountNetParameters)
    
    BoxCountEncoder.to(device)
    pixelwise_loss.to(device)

    # Optimizers
    optimizer_BC = torch.optim.Adam(BoxCountEncoder.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    if device=="cuda":
        Tensor = torch.cuda.FloatTensor 

    else:
        Tensor = torch.FloatTensor

    from tqdm import tqdm

    # ----------
    #  Training
    # ----------

    epochs =range(opt.n_epochs)

    print("Model: ",Modelname)

    for epoch in epochs:
        if verbosity: print("epoch ",str(epoch), "of", str(opt.n_epochs) ) 
            
        for i, (images, labels_2, labels_4, labels_8, labels_16 ) in enumerate(trainDataloader):
            #real_labels_2, real_labels_4, real_labels_8, real_labels_16 = labels
            
            if verbosity: print("Nr",str(i) ,"of", str(len(trainDataloader))) 
            real_labels_2 = Variable(labels_2.type(Tensor))
            real_labels_2.to(device)

            real_labels_4 = Variable(labels_4.type(Tensor))
            real_labels_4.to(device)

            real_labels_8 = Variable(labels_8.type(Tensor))
            real_labels_8.to(device)

            real_labels_16 = Variable(labels_16.type(Tensor))
            real_labels_16.to(device)

            # Configure input
            real_imgs = Variable(images.type(Tensor))
            real_imgs.to(device)

            optimizer_BC.zero_grad()

            BCR_LAK_map_2 , BCR_LAK_map_4 , BCR_LAK_map_8 , BCR_LAK_map_16 = BoxCountEncoder(real_imgs)


            BCR_LAK_map_2_loss  = pixelwise_loss(BCR_LAK_map_2, real_labels_2)
            BCR_LAK_map_4_loss = pixelwise_loss(BCR_LAK_map_4, real_labels_4)
            BCR_LAK_map_8_loss = pixelwise_loss(BCR_LAK_map_8, real_labels_8)
            BCR_LAK_map_16_loss = pixelwise_loss(BCR_LAK_map_16, real_labels_16)
            
            Scalefactor_2 , Scalefactor_4 , Scalefactor_8 , Scalefactor_16  =  0.25, 0.25, 0.25 , 0.25      # maybe optimiziing usage?!
            BCR_LAK_loss =  Scalefactor_2 * BCR_LAK_map_2_loss + Scalefactor_4 * BCR_LAK_map_4_loss + Scalefactor_8 * BCR_LAK_map_8_loss + Scalefactor_16 * BCR_LAK_map_16_loss
            if verbosity: print("loss: BCR_LAK_loss",BCR_LAK_loss) 

            #input()
            BCR_LAK_loss.backward()
            optimizer_BC.step()
        '''
        print(
            "[Epoch %d/%d] [Batch %d/%d] [BC loss: %f]  "
            % (epoch, opt.n_epochs, i, len(trainDataloader), BCR_LAK_loss.item() )
        )
        '''

    ### SAVE MODEL IF its better than 0something
    if previous_Best_Loss == None:
        previous_Best_Loss = BCR_LAK_loss.item()
    else:
        pass


    Loss_Now = BCR_LAK_loss.item()
    print("Best loss so far  :", previous_Best_Loss)
    print("loss of this model:", Loss_Now)

    if Loss_Now <= previous_Best_Loss:
        #<= to save first model always and then just, when better model was found with LOWER LOSS 
        saveplace = FileParentPath
        saveplace +="/models/"
        saveplace += "Loss" + str(round(BCR_LAK_loss.item(),3)) +"---"
        saveplace += Modelname
        NetParametersSaveplace = saveplace +".netparams"
        with open(NetParametersSaveplace, "wb") as f:
            pickle.dump(BoxCountNetParameters, f)
        
        saveplace += ".model"
        torch.save(BoxCountEncoder.state_dict(), saveplace)
        #only update, when it was higher
        print("Model Saved")
        previous_Best_Loss = Loss_Now

    else:
        print("Loss was higher/worse than previous best model")

    return {'loss': BCR_LAK_loss.item(), 'status': STATUS_OK}



### 3.5.7 Training with hyperoptimization and progress saving

To save occasional progress the trials object from the hyperparameteroptimization library can be saved and reloaded. The trials object contains the previous hyperparameters and their respective losses. The "tpe suggest" algorithm will choose the new hyperparameters, considering the content of the trials object.
The hyperparameter optimization will be initialized at first by creating a new trials object, then train three times and then saves the trials object. After loading the saved trials object, it will be saved after every hyperparameter optimization step minimizing risk of losing progress. 

Hyperopt's parameter space is simply defined by a dictionary with its chosen parameters and are ranged binary, integer like or continuously by their respective functions.

In [ ]:
#Source: [22] https://github.com/hyperopt/hyperopt/issues/267

youwanttotrain = input("Do you want to train? Y/n")

if youwanttotrain == "" or youwanttotrain == "y" or youwanttotrain == "Y":
    #To save trials object to pick up where you left
    def run_trials(HyperParameterspace, Modelname):
        #ATTENTION: If you want to begin training anew, then you have to delete the .hyperopt file
        TrialsSaveplace = FileParentPath
        TrialsSaveplace +=  "/"+ str(Modelname) +".hyperopt" 
        trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after every iteration
        max_trials = 3  # initial max_trials. put something small to not have to wait

        
        try:  # try to load an already saved trials object, and increase the max
            trials = pickle.load(open(TrialsSaveplace, "rb"))
            print("Found saved Trials! Loading...")
            max_trials = len(trials.trials) + trials_step
            print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
        except:  # create a new trials object and start searching
            trials = Trials()

        
        lowest_loss = fmin(TrainSpacialBoxcount_with, HyperparameterAndENCODERCLASS, algo=tpe.suggest, max_evals=max_trials, trials=trials)

        print("Lowest achieved loss so far:", lowest_loss)
        
        # save the trials object
        with open(TrialsSaveplace, "wb") as f:
            pickle.dump(trials, f)


    HyperParameterspace = {
        'n_epochs':hp.choice('opt.n_epochs', range(5,150,5) ),
        'batch_size':hp.choice('opt.batch_size', [2,4,8,16,32,64,128,256,512] ),
        'lr':hp.uniform('lr', 0.0000001 , 0.1 ),
        'b1':hp.uniform('b1', 0.01 , 1.0 ),
        'b2':hp.uniform('b2', 0.01 , 1.0 ),
        'Inter1':hp.choice('Inter1', range(1,512) ),
        'Inter2':hp.choice('Inter2', range(1,512) ),
        'Inter3':hp.choice('Inter3', range(1,512) ),

    }     

    HyperparameterAndENCODERCLASS = HyperParameterspace,BoxCountEncoder

    print("Begin HyperparameterOptimization")

    # loop indefinitely and stop whenever you like by setting MaxTrys
    MaxTrys = 100
    for TotalTrials in range(MaxTrys):
        Modelname = "SpacialBoxcountEncoder"+"_trialsOBJ"
        run_trials(HyperparameterAndENCODERCLASS, "BoxcountEncoder")    


###  3.5.8 Validation by testing different models
Testing against a different unseen dataset is essential to prevent the model from just memorizing the incoming data also known as overfitting. 
After choosing a saved model by its modelname, the network parameters and the the models state dictionary are loaded to initialize the model. Changing the device-string, the computation place of the model can be chosen to compare the total computation time between cpu or gpu driven networks. While testing, the model is loaded in evaluation mode which disables any adjustments of the models weights and biases.

Models, which need to be tested can be chosen by appending the wanted modelname, located in the subfolder 'models', to the modelname list.
The test will load the models sequentially to iterate while tracking the mean loss and an approximation of data throughput by processed input pictures in megapixel per second.
To compare the cpu and gpu driven models, tests on both devices are executed.
To measure time, set the variable verbosity to false. To render images, neural network output and for debugging code, set verbosity to true.

In [ ]:
# ----------
#  Testing BoxcountEncoder
# ----------
ModelnameList = []
#Pretrained Networks-----------------------
ModelnameList.append("Loss0.01---n_epochs_135_batch-size_4_learning-rate_0.001_beta-decay_0.671_0.362")
ModelnameList.append("Loss0.01---n_epochs_85_batch-size_512_learning-rate_0.001_beta-decay_0.681_0.876")
ModelnameList.append("Loss0.014---n_epochs_5_batch-size_128_learning-rate_0.001_beta-decay_0.501_0.945")
ModelnameList.append("Loss0.506---n_epochs_25_batch-size_512_learning-rate_0.071_beta-decay_0.26_0.248")
ModelnameList.append("Loss0.735---n_epochs_95_batch-size_4_learning-rate_0.062_beta-decay_0.311_0.194")
ModelnameList.append("Loss0.012---n_epochs_135_batch-size_4_learning-rate_0.015_beta-decay_0.808_0.762")

showitem = None
whereTObreakIteration = 100   #at batch the test will break the testloop to continue


#To render the network output set verbosity to True
verbosity = False
opt.n_cpu = 8 #for every thread of my quadcore -> adjust as you like

def TestSpacialBoxcount_with(Modelname, BoxCountEncoder,showitem, device):
    #global showitem
    NetParametersSaveplace =FileParentPath+ "/models/"+ Modelname +".netparams"
    BoxCountNetParameters = pickle.load(open(NetParametersSaveplace, "rb"))

    saveplace = FileParentPath+ "/models/"+Modelname +".model"
    
    __, parameter = Modelname.split('---')  #extracting parameter to generate option object
    __, __, n_epochs, __,   batch_size, __, learning_rate, __ , betadecay1, betadecay2 = parameter.split('_')
    
    
    #should not be nessecary, except the changing batchsize #self, n_epochs, batch_size, img_size, channels, learning_rate, b1, b2
    opt = OptionObject(int(n_epochs), int(batch_size), ChunkLenght, 1 , float(learning_rate), float(betadecay1), float(betadecay2))
    
    #device = "cuda"
    #device = "cpu"
    
    #load and init the BCencoder
    BoxCountEncoder = BoxCountEncoder(BoxCountNetParameters)
    BoxCountEncoder.load_state_dict(torch.load(saveplace, map_location=device))
    BoxCountEncoder.eval()   #to disable backpropagation, so don't adjust any weights and biases

    #define Loss
    pixelwise_loss = torch.nn.L1Loss()

    #Init BoxcountEncoder
    BoxCountEncoder.to(device)
    pixelwise_loss.to(device)

    # Optimizers
    optimizer_BC = torch.optim.Adam(BoxCountEncoder.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    if device=="cuda":
        Tensor = torch.cuda.FloatTensor 
    else:
        Tensor = torch.FloatTensor
    
    totaltime = 0.0
    running_loss = 0.0
    
    
    #Begin testing by evaluating the test data set
    for  i, (images, labels_2, labels_4, labels_8, labels_16 )  in enumerate(testDataLoader):
        if i== whereTObreakIteration:
            print("reached iteration",whereTObreakIteration, "break loop to test next model" )
            break
            
        start = time.time()
        
        torch.no_grad() # for testing no gradients have to be computed
        # Configure input/ouput variables and send it to device
        real_imgs = Variable(images.type(Tensor))
        real_imgs.to(device)
        # BCR_map/Lac_map with boxsize 2
        real_labels_2 = Variable(labels_2.type(Tensor))
        real_labels_2.to(device)
        # BCR_map/Lac_map with boxsize 4
        real_labels_4 = Variable(labels_4.type(Tensor))
        real_labels_4.to(device)
        # BCR_map/Lac_map with boxsize 8
        real_labels_8 = Variable(labels_8.type(Tensor))
        real_labels_8.to(device)
        # BCR_map/Lac_map with boxsize 16
        real_labels_16 = Variable(labels_16.type(Tensor))
        real_labels_16.to(device)

        BCR_LAK_map_2 , BCR_LAK_map_4 , BCR_LAK_map_8 , BCR_LAK_map_16 = BoxCountEncoder(real_imgs)
        optimizer_BC.zero_grad()
        
        #necessary for accessing the picture again with numpy
        NumpyencImg2 =  BCR_LAK_map_2.cpu().detach().numpy()
        NumpyencImg4 =  BCR_LAK_map_4.cpu().detach().numpy()
        NumpyencImg8 =  BCR_LAK_map_8.cpu().detach().numpy()
        NumpyencImg16 =  BCR_LAK_map_16.cpu().detach().numpy()

        if verbosity:
            #Render Pictures 
            for idx in range(opt.batch_size):
                print("index within batch:", idx)
                showNPArrayAsImage(images[idx,0,:,:], "Original Image", "gray")

                #source: [18] https://stackoverflow.com/questions/22053274/grid-of-images-in-matplotlib-with-no-padding
                max_cols = 8
                fig, axes = plt.subplots(nrows=2, ncols=max_cols, figsize=(16,4))
                lablelist = [labels_2[idx,0,:,:], labels_2[idx,1,:,:], labels_4[idx,0,:,:], labels_4[idx,1,:,:], labels_8[idx,0,:,:], labels_8[idx,1,:,:], labels_16[idx,0,:,:],  labels_16[idx,1,:,:], NumpyencImg2[idx,0,:,:], NumpyencImg2[idx,1,:,:], NumpyencImg4[idx,0,:,:], NumpyencImg4[idx,1,:,:], NumpyencImg8[idx,0,:,:], NumpyencImg8[idx,1,:,:], NumpyencImg16[idx,0,:,:]  , NumpyencImg16[idx,1,:,:]]
                #titlelist = ["BCR2", "LAC2", "BCR4", "LAC4", "BCR8", "LAC8","BCR16", "LAC16", "NN BCR2", "NN LAC2",   ]
                ylabellist = ["CPU", "", "","", "", "", "","", "GPU", "","","","", "", "",""]
                xlabellist = ["", "", "","", "", "","","","BCR2" ,"LAC2" ,"BCR4" ,"LAC4" ,"BCR8" ,"LAC8" ,"BCR16" ,"LAC16" ]
                for idx, image in enumerate(lablelist):
                    row = idx // max_cols
                    col = idx % max_cols
                    #axes[row, col].axis("off")
                    axes[row, col].imshow(image, cmap="gray", aspect="auto")
                    #axes[row, col].set_title(titlelist[idx])
                    # label x-axis and y-axis 
                    axes[row, col].set_ylabel(ylabellist[idx]) 
                    axes[row, col].set_xlabel(ylabellist[idx]) 

                plt.subplots_adjust(wspace=.05, hspace=.05)
                plt.show()

                if showitem == None or showitem == "y" or showitem == "Y":
                    showitem = input("press  y/Y for next item in batch  or else to continue with next batch")
                    if showitem == "y" or showitem == "Y":
                        continue
                    else:
                        showitem = None
                        break

                    
        end = time.time()  #cause loss calulation has nothing to do with boxcount calc time

        # Scalable pixelwise loss to have    
        BCR_LAK_map_2_loss  = pixelwise_loss(BCR_LAK_map_2, real_labels_2)
        BCR_LAK_map_4_loss = pixelwise_loss(BCR_LAK_map_4, real_labels_4)
        BCR_LAK_map_8_loss = pixelwise_loss(BCR_LAK_map_8, real_labels_8)
        BCR_LAK_map_16_loss = pixelwise_loss(BCR_LAK_map_16, real_labels_16)
        Scalefactor_2 , Scalefactor_4 , Scalefactor_8 , Scalefactor_16  =  0.25, 0.25, 0.25 , 0.25      # maybe optimiziing usage?!

        BCR_LAK_loss =  Scalefactor_2 * BCR_LAK_map_2_loss + Scalefactor_4 * BCR_LAK_map_4_loss + Scalefactor_8 * BCR_LAK_map_8_loss + Scalefactor_16 * BCR_LAK_map_16_loss
        
        running_loss += BCR_LAK_loss.item()

        BCR_LAK_loss.backward()
        optimizer_BC.step()
        
        timePERbatch = end - start
        totaltime += timePERbatch
        
        if verbosity: 
            print("[Batch %d/%d] [BC loss: %f] "% ( i, len(testDataLoader), BCR_LAK_loss.item()))
            print(timePERbatch, " seconds for boxcounting 1 file with batch_size of",opt.batch_size )
            
        #input("presskey fornext batch")
    mean_timePERbatch = totaltime / float(whereTObreakIteration)
    mean_loss = running_loss/ float(whereTObreakIteration)  # cause testing will abort after 100 batchesm has to be normalized 

    return mean_loss, mean_timePERbatch

youwanttotest = input("Do you want to test? Y/n")


scoreboard = {}
#{'Modelname': mean_loss, timePerbatch, MegapixelPERsecond}

#If cuda is avaiable, then test against both
device = get_device()
if device=="cuda":
    devicelist = ['cpu','cuda']
else:
    devicelist = ['cpu']

if youwanttotest == "" or youwanttotest == "y" or youwanttotest == "Y":
    for device in devicelist:
        
        for Modelname in ModelnameList:
            print("-----------begin test with new model-------------")
            print("Chosen Device is", device)
            print("Modelname: ", Modelname)
            mean_loss,  mean_timePERbatch = TestSpacialBoxcount_with(Modelname,BoxCountEncoder,showitem, device)
            mean_loss,  mean_timePERbatch = round(mean_loss,2),  round(mean_timePERbatch,2) 
            MegapixelPERsecond =  round( (opt.batch_size * opt.img_size**2) /(mean_timePERbatch* 1000000 ) ,2)
            fullmodelname = device +'_'+ Modelname
            scoreboard[fullmodelname] = [mean_loss, mean_timePERbatch, MegapixelPERsecond ]
            print("This model performed with a mean loss of", mean_loss, "with a mean time/batch", mean_timePERbatch, "with a pixelthroughput of",MegapixelPERsecond," Mpx/s" )

    #print("scoreboard: ", scoreboard)
    for key, item in scoreboard.items():
        print("Model:",key," with mean_testloss of", item[0], " with a mean time/batch", item[1], "with a pixelthroughput of", item[2] )


## 4 Results and discussion

### 4.1 Results:

A spacially depending boxcounting algorithm was proposed, where a box scans in a fixed grid manner over a slice of a 3D array to count the filled boxes in comparison to the total number of boxes, named the boxcount ratio. It also counts the number of datapoints per cube to be able to calculate the spacial heterogeneity, also known as the lacunarity.
So instead of counting boxes for the whole picture at once the picture was chunked into chunks in the range of all chosen boxsizes, to spacially characterize the input picture at different scales. 
In every chunk ranging from chosen borders in x and y, but the whole z-direction the boxcounting was executed and a boxcount ratio and lacunarity returned to ensemble a two channel scaled down array of the picture like in figure 10 demonstrated. 


![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/generated_imgs/7_4_3700x.png)

Figure 10: Generated output from the spacial boxcounting with the spacial boxcount ratio as BCR and lacunarity as LAC with their boxsizes 2, 4, 8, 16

The boxcount ratio is an indicator of spacial complexity. From these boxcounts ranging over different boxsizes, the fractal dimension could be calculated, which gives any object a continuous dimensionality.
The lacunarity is an indicator for spacial heterogeneity, so it detects defects in patterns. By changing the boxsize the lacunarity can be controlled in terms of scaling, so defects or patterns at different scales are adressed to make the lacunarity a candidate for an edge detection application, or for dimensionality reduction as a preprocessing step.

The boxcount algorithm can be split up during execution into different threads handling all different boxsizes at once to gain some speed compared to single threaded performance, but consumes also more system memory. 

Boxcounting allows for quantitative continuous characterization of arbitrary 2D arrays for statistical analysis, spacial categorizing, similarity search and sorting purposes.

Spacial boxcounting retains spacial information by returning an array of boxcount ratio and lacunaritys, where any value of this resulting array the spacial complexity or heterogeneity in a given point describes.

To harness potential parallel processing power an attempt was made to translate the previous cpu driven process into a neural network. This network allows for all the processing for all boxsizes at once, so it could be faster then the sequential driven process, even with multithreading, depending on the chosen hardware.


###  4.2 Discussion

#### 4.2.1 Timebased computational efficiency  

The time elapsed per processed picture with the spacial boxcount from 3.3.2 is heavily dependend on the chosen boxsize. So by increasing the minimal boxsize, the execution time per picture but also the resolution for finest pattern decreases. 
If the spacial boxcounting is executed just with one boxsize, the process takes 1-2 seconds with the boxsize 2 and just 100 to 10 milliseconds for bigger boxsizes. Beware, that the maximum boxsize can only be as big as the minimum value of the pictures resolution or depth of color.

By measuring the time taken to execute and return the wanted values, the computational time efficiency can be observed. Since this demonstration uses pictures, the metric megapixels of processed, incoming data per second is used.
A grayscale full HD video with 30 frames per second would result in  62,20 megapixel per second, which would allow for realtime video analysis or processing.
The cpu driven multithreaded process with the standard python interpreter or the numba-jit compiler can be compared against the neural network loaded into the cpu or gpu.
Here the intel i7-4790k cpu's performance  is compared to a nvidia gtx 1070. So execution times may vary on different systems. The time for each category was measured over 100 pictures for the boxcounting and 100 batches for the network.

Note, that the neural network does never achieve a loss of zero, so the quality of the results is always lower than the original boxcount algorithm. 

The standard python interpreter is by far the worst, measured at a mean of 93.67 seconds per picture sized 1280 x 960 pixels with a processing speed of 0.03 megapixel per second.

To keeping the code applicable to the numba-jit compiler, by isolating computational heavy tasks into conform functions and decorating the functions with "@jit(nopython=True)" to enable a huge gain in computational efficiency resulting in a mean of 1.46 seconds per pictures with 0.84 megapixel per second. This is a performance gain of 28 times compared to the standard python interpreter in terms of processed data over time.

Training a neuronal network is computational intensive so a gpu might be needed. The models were loaded in each instance and measured slightly less performant for the neural network executed on the cpu with values ranging from 1.94 to 8.74 megapixel per second, as of on the gpu ranging from 8.74 up to 52.43 megpixel per second. This indicates, that the gpu can utilize parallel processing power, even when the data has to be transmitted from system memory to graphics memory. With the maximum performing model, the gpu would be able to process video or streams in realtime, but with loss of accuracy.

The fastest model is named "cuda_Loss0.506---n_epochs_25_batch-size_512_learning-rate_0.071_beta-decay_0.26_0.248" with 52.43 megpixel per second on the gpu with a mean test loss of 0.53.


#### 4.2.2 Loss comparison

The most accurate model was named "cuda_Loss0.01---n_epochs_135_batch-size_4_learning-rate_0.001_beta-decay_0.671_0.362" with a mean test loss of 0.01 with 26.21 megapixel/second of processed images, if executed on the gpu.

![image info](https://raw.githubusercontent.com/ollimacp/spacial-boxcounting-cpu-gpu/main/0Data/Document_images/Accuracy_comparison.png)

Figure 11: Input picture and output from spacial boxcounting algorithm and neural network output  

Figure 11 shows a particular output of an input image. The first row shows the results of the spacial boxcounting in every boxsize. The second row displays the output of the neural network. The loss of 0.01 seems little compared to the fastest model with a mean test loss of 0.53, but by comparing the network generated output against the cpu driven process reveals discrepancies. The generated output seems to have some kind of similarity, but the observed differences are indisputable. The actual neural network architecture is not optimal and accurate enough to replace the cpu driven process for faster computation at this point in time.
To decrease the loss further, additional convolutional neural layers could be implemented, or batch normalization layers and potentially other activation functions could be utilized or just more data could be used to train the neural network.



## 5 Conclusion

Boxcounting is able to quantitatively characterize any kind of data in matrix form and describes spacial complexity at different scalings. The scaling is described by the boxsize and should be executed with multiple sizes to gain scale specific knowledge. The lacunarity or spacial heterogeneity can be calculated while revealing scale depend defects in homogeneous patterns.

The spacial boxcounting algorithm chunks the incoming image into boxsized pieces and calculates the spacial complexity and heterogeneity  at any point in the image, enabling spacial characterization of data. The input images can be compared to another, enabling sorting in complexity or heterogeneity, similarity search or pattern defect detection.

While development time of python programs is faster than low-level programming  languages, the execution time of python code is terrible. By using a high performance compiler, the computation time can be many times faster, than the standard python compiler. 

To gain more time based performance, a convolutional neural network with multiple outputs were used to mimic the cpu driven process and try to compute every output in one forward pass. The computation time per picture can be lowered even more, but the outputs of the neural network are not completely accurate, so decreased quality is traded for increased quantity. Further optimizing the models architecture can resolve in a model fast and accurate enough to enable realtime video or stream processing.

## 6 ACKNOWLEDGMENTS AND SOURCES

### Example dataset location: <https://drive.google.com/file/d/1sLc8Uk61W_4TTtmRk4x6_5cOiWq7IeOT/view?usp=sharing>

### [1] Boxcounting:  <https://en.wikipedia.org/wiki/Box_counting>

### [2] Lacunarity:  <https://en.wikipedia.org/wiki/Lacunarity>

### [3] Numpy documentation: <https://numpy.org/doc/1.20/>

### [4] TQDM  documentation: <https://tqdm.github.io/docs/tqdm/>

### [5] Matplotlib  documentation: <https://matplotlib.org/3.3.3/contents.html>

### [6] Pathlib  documentation: <https://docs.python.org/3/library/pathlib.html>

### [7] Numba  documentation:  <https://numba.readthedocs.io/en/stable/user/jit.html>

### [8] Pytorch   documentation: <https://pytorch.org/docs/stable/nn.init.html>

### [9] Sci-kit learn  documentation: <https://sklearn.org/documentation.html>

### [10] Hyperopt documentation: <http://hyperopt.github.io/hyperopt/>

### [11] OS libary documentation:  <https://docs.python.org/3/library/os.html>

### [12] Pickle documentation:  <https://docs.python.org/3/library/pickle.html>

### [13] Time documentation: <https://docs.python.org/3/library/time.html>

### [14] Pillow image libary: <https://pillow.readthedocs.io/en/stable/>

### [15] How to install pytorch : <https://varhowto.com/install-pytorch-1-6-0/>

### [16] Exception handeling: <https://stackoverflow.com/questions/14519177/python-exception-handling-line-number#20264059>

### [17] Custom threading base class:  <https://stackoverflow.com/questions/6893968/how-to-get-the-return-value-from-a-thread-in-python>

### [18] Display images with matplotlib:  <https://stackoverflow.com/questions/22053274/grid-of-images-in-matplotlib-with-no-padding>

### [19] Sort dictionary by value: <https://stackabuse.com/how-to-sort-dictionary-by-value-in-python/>

### [20] Pad numpy array with zeros:  <https://stackoverflow.com/questions/35751306/python-how-to-pad-numpy-array-with-zeros>

### [21] Create custom dataset for pytorch: <https://stackoverflow.com/questions/56774582/adding-custom-labels-to-pytorch-dataloader-dataset-does-not-work-for-custom-data>

### [22] Save progress during hyperoptimization: <https://github.com/hyperopt/hyperopt/issues/267>


